In [22]:
import numpy as np
import pandas as pd
import warnings,gc,os,time,math
import pymysql
warnings.filterwarnings('ignore')
from dateutil.relativedelta import relativedelta
from datetime import datetime
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import lightgbm as lgb
import pickle

In [23]:
filename='ud20201019'
num='_34' 
today='20201023'

datap='/home/jupyter/LQ/VW/data/'
new_datap='/home/jupyter/LCG/dazhong/V2/%s/'%(filename) #读取阿桂的数据
savep='/home/jupyter/LQ/VW/20191217_VW_下次保养里程预测/20201023自动化更新数据/%s%s/'%(today,str(num))

if os.path.exists(savep):
    print('True')
else:
    os.mkdir(savep)
    
store_num=num.replace('_','')
store_num=int(store_num)
print(store_num)

if (store_num<=20):
    baoyang_id_max=10
elif (store_num>20)&(store_num<=40):
    baoyang_id_max=11
elif (store_num>40)&(store_num<=50):
    baoyang_id_max=12
elif store_num>50:
    baoyang_id_max=13
    
print('biggest batch :',baoyang_id_max)

True
30
biggest batch : 11


In [3]:
def check_df(df):
    df=df.drop_duplicates()
    print(df.shape)
    print(df[['VIN','修理日期']].drop_duplicates().shape)
    print(df['VIN'].nunique())
    
def check_run_time(s):
    r=(time.time()-s)/60
    return "Run Mins: %f"%r

def get_car(df,old_name):
    df[old_name]=df[old_name].apply(lambda x:str(x).lower())
    pattern='辉昂|Phideon|phideon'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='辉昂'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='途昂|Teramont|teramont'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='途昂'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='帕萨特|Passat|passat|新领驭|领驭'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='帕萨特'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='途观|Tiguan|tiguan'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='途观'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='凌渡|Lamando|lamando'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='凌渡'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='朗行|朗逸|朗境|Lavida|lavida'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='朗逸'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='途安|Touran|touran'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='途安'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='Polo|polo|波罗'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='Polo'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='桑塔纳|Santana|santana'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='桑塔纳'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='途岳|tharu|Tharu'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='途岳'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='途铠|t-cross'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='途铠'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    df.loc[df['二级车系'].isnull(),'二级车系']='非电动车'
    return df


def get_car_family(df,new_name):
    high=['辉昂','途昂']
    mid_high=['帕萨特','帕萨特电动','途观','途观电动']
    mid=['凌渡','朗逸','朗逸电动','途安','途岳']
    low=['Polo','桑塔纳','途铠']
    df.loc[df[new_name].isin(high),'车型档次']='高档'
    df.loc[df[new_name].isin(mid_high),'车型档次']='中高档'
    df.loc[df[new_name].isin(mid),'车型档次']='中低档'
    df.loc[df[new_name].isin(low),'车型档次']='低档'
    return df


def get_miss_cols(df):
    tt=pd.isnull(df).sum().reset_index()
    tt.columns=['字段名','缺失值']
    tt['缺失值占比']=tt['缺失值']/df.shape[0]
    tt=tt.sort_values(by=['缺失值占比'],ascending=False).reset_index(drop=True)
    miss_cols=tt[tt['缺失值占比']>0]['字段名'].unique().tolist()
    miss_df=tt[tt['缺失值占比']>0]
    return miss_cols,miss_df

def get_last_run(df):
    df['修理日期']=pd.to_datetime(df['修理日期'])
    df['购车时间']=pd.to_datetime(df['购车时间'])
    df=df.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)
    df['修理日期_last']=df.groupby('VIN')['修理日期'].shift(-1)
    df['公里数_last']=df.groupby('VIN')['公里数'].shift(-1)
    df.loc[df['修理日期_last'].isnull(),'修理日期_last']=df['购车时间']
    df.loc[df['公里数_last'].isnull(),'公里数_last']=0
    df['天数差']=(df['修理日期']-df['修理日期_last']).apply(lambda x:x.days)
    df['公里数差']=(df['公里数']-df['公里数_last'])
    df['日均差']=df['公里数差']/df['天数差']
    return df


def get_user(df):
    df['VIN']=df['VIN'].apply(str)
    df['brand']=df['VIN'].apply(lambda x:str(x)[:3])
    df['车身型式']=df['VIN'].apply(lambda x:x[3])
    df['变速器']=df['VIN'].apply(lambda x:x[4])
    df['乘员保护系统']=df['VIN'].apply(lambda x:x[5])
    df['车辆等级']=df['VIN'].apply(lambda x:x[6:8])
    df['生产年份']=df['VIN'].apply(lambda x:x[9])
    df['装配厂']=df['VIN'].apply(lambda x:x[10])
    for col in ['车身型式', '变速器', '乘员保护系统', '车辆等级', '生产年份', '装配厂']:
        df[col]=df[col].astype(str)
    df['用户属性']=list(map(lambda a,b,c,d,e,f:a+','+b+','+c+','+d+','+e+','+f,df['车身型式'],df['变速器'],
                        df['乘员保护系统'],df['车辆等级'],df['生产年份'],df['装配厂']))
    return df

In [4]:
print("仅为6个固定字段,补充缺失值")
def fill_missing(df,by_col,car_rel):

    if '发动机组号' not in list(df.columns):
        df.rename(columns={'发动机-机组号':'发动机组号'},inplace=True)
    if '变速箱机组号' not in list(df.columns):
        df.rename(columns={'变速箱-机组号':'变速箱机组号'},inplace=True)
        
    del_col=[]
    for col in car_rel:
        tmp=df.groupby([by_col,col])['VIN'].nunique().reset_index(name='人数')
        tmp=tmp.sort_values(by=[by_col,'人数'],ascending=False).drop_duplicates(subset=[by_col],keep='first').reset_index(drop=True)
        del tmp['人数']
        tmp.columns=[by_col,'%s_众数'%col]
        df=df.merge(tmp,on=[by_col],how='left')
        df.loc[df[col].isnull(),col]=df['%s_众数'%col]
        df.loc[df[col]=='nan',col]=df['%s_众数'%col]
        del_col.append('%s_众数'%col)
        gc.collect()
    df1=df.drop(del_col,axis=1)
    return df1

仅为6个固定字段,补充缺失值


In [5]:
def get_final_baoyang(baoyang,veh,datap):
    del veh['购车时间']
    del veh['新购车时间']
    
    revise_types=['首保','常规保养','附件安装','一般维修','事故车维修','普通索赔','服务活动',
                  '二手车认证维修','内部车辆维修']
    for col in revise_types:
        baoyang[col]=baoyang[col].fillna(0)

    if ('修理类型' in list(baoyang.columns)) and ('修理类型_new' in list(baoyang.columns)):
        del baoyang['修理类型']
        baoyang.rename(columns={'修理类型_new':'修理类型'},inplace=True)
    
    baoyang=get_last_run(baoyang)

    del_cols=[x for x in list(baoyang.columns) if x in list(veh.columns)]
    del_cols.remove('VIN')

    baoyang=baoyang.drop(del_cols,axis=1)

    baoyang=baoyang.merge(veh,on=['VIN'],how='left')
    baoyang=get_user(baoyang)

    if 'baoyang_id' in list(baoyang.columns):
        del baoyang['baoyang_id']
    baoyang['baoyang_id']=baoyang.groupby('VIN')['修理日期'].rank(method='dense',ascending=True)

    baoyang['天数']=(baoyang['修理日期']-baoyang['购车时间']).apply(lambda x:x.days)
    baoyang['日均']=baoyang['公里数']/baoyang['天数']
    
    if '保养次数' in list(baoyang.columns.tolist()):
        del baoyang['保养次数']
    if '历史保养次数' in list(baoyang.columns.tolist()):
        del baoyang['历史保养次数']
    tmp=baoyang.groupby('VIN')['修理日期'].nunique().reset_index(name='保养次数')
    baoyang=baoyang.merge(tmp,on=['VIN'],how='left')
    
    print("最后检查是否还有缺失值")
    mis1,mis2=get_miss_cols(baoyang)
    print(mis1)
    
    tmpp=baoyang.groupby('保养次数')['VIN'].nunique().reset_index(name='人数')
    baoyang_id_max=tmpp[tmpp['人数']>=4000]['保养次数'].max()
    baoyang_id_max=baoyang_id_max+1
    print('人数>=4000人的最大历史保养次数是:',baoyang_id_max-1)
    print('人数<4000人的最小历史保养次数是:',baoyang_id_max)
    print('其中,baoyang_id_max就是最后一个batch,即np.range(2,baoyang_id_max+1,1)')
    
    return baoyang,baoyang_id_max

In [6]:
def get_xun(d):
    if d<=10:
        return 1
    elif (d>10) and (d<=20):
        return 2
    elif d>20:
        return 3

def get_next_label_run(df):
    df=df.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)
    df['修理日期_next']=df.groupby('VIN')['修理日期'].shift(1)
    df['公里数_next']=df.groupby('VIN')['公里数'].shift(1)
    return df

def get_all_feat_eng(baoyang7,today):
    baoyang7['购车时间']=pd.to_datetime(baoyang7['购车时间'])
    baoyang7['修理日期']=pd.to_datetime(baoyang7['修理日期'])
    baoyang7=baoyang7.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)
    baoyang7['buy_year']=baoyang7['购车时间'].dt.year
    baoyang7['buy_mon']=baoyang7['购车时间'].dt.month
    baoyang7['车龄bin']=(pd.to_datetime(today)-baoyang7['购车时间']).apply(lambda x:math.ceil(x.days/180))
    minn=baoyang7['buy_year'].min()
    baoyang7['buy_id']=list(map(lambda x,y:(x-minn)*12+y,baoyang7['buy_year'],baoyang7['buy_mon'])) #精确到年月

    baoyang7['year']=baoyang7['修理日期'].dt.year
    baoyang7['mon']=baoyang7['修理日期'].dt.month

    baoyang7['day']=baoyang7['修理日期'].dt.day
    baoyang7['xun']=baoyang7['day'].apply(get_xun)
    baoyang7['weekdays']=baoyang7['修理日期'].apply(lambda x:datetime.strftime(x,'%w'))
    baoyang7['is_workday']=baoyang7['weekdays'].apply(lambda x:1 if x not in ['0','6'] else 0)

    baoyang7=get_next_label_run(baoyang7)
    for col in ['公里数_last','修理日期_last']:
        del baoyang7[col]

    return baoyang7

In [7]:
def get_win_data(df,baoyang_id_num,baoyang_id_max):
    if (baoyang_id_num>=2) and (baoyang_id_num<baoyang_id_max):
        dt1=df[df['保养次数']>=baoyang_id_num]
        train1=dt1[dt1['baoyang_id']<=(baoyang_id_num-1)]
        test1=df[df['保养次数']==(baoyang_id_num-1)]
        return train1,test1
    
    elif baoyang_id_num<2:
        return [],[]
    
    elif baoyang_id_num==baoyang_id_max:
#         print('注意这里有一个bug,历史保养次数10次的人应该去预测历史保养次数9次的,所以实际上历史保养9次及以上的人还没有被预测,打包放到这个batch中一起训练')
        dt1=df[df['保养次数']>=(baoyang_id_num-1)]
        if 'baoyang_id' in list(dt1.columns):
            del dt1['baoyang_id']
        dt1['baoyang_id']=dt1.groupby('VIN')['修理日期'].rank(method='dense',ascending=False)
        maxx=dt1['保养次数'].max()
        train1=dt1[dt1['baoyang_id'].between(2,maxx)] #训练集中去掉最后一次,因为是倒序,所以最后一次就是1
        train1['数据集标注']='训练集'
        test1=dt1.copy() #测试集全放
        test1['数据集标注']='测试集'
        return train1,test1

    
def get_right_data(df,baoyang_id_num,baoyang_id_max):
    if baoyang_id_num>=2:
        train1,test1=get_win_data(df,baoyang_id_num,baoyang_id_max)
        train1_label=train1[['VIN','修理日期_next','公里数_next']].sort_values(by=['VIN','修理日期_next'],ascending=False).drop_duplicates(['VIN'],keep='first').reset_index(drop=True)
    
        del train1['修理日期_next']
        del train1['公里数_next']
        train1=train1.merge(train1_label,on=['VIN'],how='left')
    
        train1['天数差_next']=(train1['修理日期_next']-train1['购车时间']).apply(lambda x:x.days)
        train1['公里数差_next']=train1['公里数_next']-0
        train1['日均_next']=train1['公里数差_next']/train1['天数差_next']
        del train1['修理日期_next']
        del train1['公里数_next']
        train1=train1.drop_duplicates().reset_index(drop=True)
#         print("检查训练集")
#         print('对应的baoyang_id_batch',baoyang_id_num)
#         print(train1.shape)
#         print(train1['VIN'].nunique())
    
        del test1['修理日期_next']
        del test1['公里数_next']
#         print("检查测试集")
#         print(test1.shape)
#         print(test1['VIN'].nunique())
        data1=pd.concat([train1,test1],axis=0)
    
    else:
        print("the baoyang id should >=2")
        
    return data1


def quchong(d):
    ss=str(d).split(',')
    lst=[]
    for a in ss:
        if a not in lst:
            lst.append(a)
    return ','.join(lst)

In [8]:
def get_the_dt_onehot(data1,the_baoyang_id,baoyang_id_max):
    train_cols1=[]
    
    data1['修理日期']=pd.to_datetime(data1['修理日期'])
    for col in ['维修金额','维修金额_折前']:
        data1[col]=data1[col].astype(float)
        
    data1['是否有折扣']=(data1['维修金额']-data1['维修金额_折前']).apply(lambda x:1 if x<-10 else 0)

    revise_types=['首保','常规保养', '附件安装', '一般维修', '事故车维修', '普通索赔', 
              '服务活动', '二手车认证维修', '内部车辆维修']
    for col in revise_types:
        data1[col]=data1[col].fillna(0)
        data1[col]=data1[col].astype(int)

    cat_feat1=['会员卡类型','性别', '一级车系', '二级车系', 'buy_year', '会员卡年龄', '会员卡状态', '换购',
           '是否绑定微信', '车龄bin', '车辆类型', '使用性质', '车主性质']
    train_cols1=train_cols1+cat_feat1
    for col in cat_feat1:
        data1[col]=data1[col].astype(str)
        dum=pd.get_dummies(data1[col],prefix=col)
        train_cols1=train_cols1+list(dum.columns)
        data1=pd.concat([data1,dum],axis=1)
    for col in cat_feat1: 
        tmp=data1.groupby(col)['VIN'].nunique().reset_index(name='cnt')
        tmp=tmp.sort_values(by=['cnt'],ascending=False).reset_index(drop=True)
        tmp['%s_le'%col]=range(len(tmp),0,-1)
        tmp_label=dict(zip(tmp[col],tmp['%s_le'%col]))
        data1[col]=data1[col].map(tmp_label)

    cat_feat2=['车型代码','省份','城市','常去经销商']
    train_cols1=train_cols1+cat_feat2
    for col in cat_feat2:
        tmp=data1.groupby(col)['VIN'].nunique().reset_index(name='cnt')
        tmp=tmp.sort_values(by=['cnt'],ascending=False).reset_index(drop=True)
        tmp['%s_le'%col]=range(len(tmp),0,-1)
        tmp_label=dict(zip(tmp[col],tmp['%s_le'%col]))
        data1[col]=data1[col].map(tmp_label)

    #不需要labelencoder,也可以onehot
    cat_feat3=['buy_id']
    train_cols1=train_cols1+cat_feat3
    for col in cat_feat3:
        data1[col]=data1[col].astype(int)
    
    cat_feat4=['车型档次','城市级别']
    train_cols1=train_cols1+cat_feat4
    for col in cat_feat4:
        data1[col]=data1[col].astype(int)
        dum=pd.get_dummies(data1[col],prefix=col)
        train_cols1=train_cols1+list(dum.columns)
        data1=pd.concat([data1,dum],axis=1)

    #需要labelencoder,也可以onehot
    cat_feat5=['车身型式', '变速器', '乘员保护系统', '车辆等级', '生产年份', '装配厂']
    train_cols1=train_cols1+cat_feat5
    for col in cat_feat5:
        data1[col]=data1[col].astype(str)
        dum=pd.get_dummies(data1[col],prefix=col)
        train_cols1=train_cols1+list(dum.columns)
        data1=pd.concat([data1,dum],axis=1)    
    for col in cat_feat5:
        tmp=data1.groupby(col)['VIN'].nunique().reset_index(name='cnt')
        tmp=tmp.sort_values(by=['cnt'],ascending=False).reset_index(drop=True)
        tmp['%s_le'%col]=range(len(tmp),0,-1)
        tmp_label=dict(zip(tmp[col],tmp['%s_le'%col]))
        data1[col]=data1[col].map(tmp_label)

    cat_feat6=['发动机组号', '变速箱机组号','进气方式','技术代码', '排量','档位']
    train_cols1=train_cols1+cat_feat6
    for col in cat_feat6:
        data1[col]=data1[col].astype(str)
        dum=pd.get_dummies(data1[col],prefix=col)
        train_cols1=train_cols1+list(dum.columns)
        data1=pd.concat([data1,dum],axis=1)
    for col in cat_feat6:
        tmp=data1.groupby(col)['VIN'].nunique().reset_index(name='cnt')
        tmp=tmp.sort_values(by=['cnt'],ascending=False).reset_index(drop=True)
        tmp['%s_le'%col]=range(len(tmp),0,-1)
        tmp_label=dict(zip(tmp[col],tmp['%s_le'%col]))
        data1[col]=data1[col].map(tmp_label)
    
#     print('############得到每次相关的特征############')
#     if the_baoyang_id<baoyang_id_max:
#         tmp=data1[['VIN','修理类型']].drop_duplicates().groupby('VIN').apply(lambda x:','.join(x['修理类型'])).reset_index(name='历史修理类型')
#         data1=data1.merge(tmp,on=['VIN'],how='left')
#         data1['历史修理类型']=data1['历史修理类型'].apply(quchong)
#     else:
#         tmp=data1[['VIN','数据集标注','修理类型']].drop_duplicates().groupby(['VIN','数据集标注']).apply(lambda x:','.join(x['修理类型'])).reset_index(name='历史修理类型')
#         data1=data1.merge(tmp,on=['VIN','数据集标注'],how='left')
#         data1['历史修理类型']=data1['历史修理类型'].apply(quchong)
    
    time_feat1=['天数','天数差','公里数','公里数差','日均','日均差']
    for col in time_feat1:
        data1[col]=data1[col].astype(float)

    time_feat2=['工时费', '维修金额', '零件费'] 
    data1['工时费']=data1['工时费'].apply(int)
    data1['零件费']=data1['零件费'].apply(int)
    data1['维修金额']=data1['工时费']+data1['零件费']

    time_feat3=['是否有折扣','year', 'mon','xun', 'weekdays', 'is_workday']
    for col in ['year', 'mon','xun', 'weekdays', 'is_workday']:
        data1[col]=data1[col].astype(int)

    time_feat4=['修理类型','经销商代码']
    for col in time_feat4:
        tmp=data1.groupby(col)['VIN'].nunique().reset_index(name='cnt')
        tmp=tmp.sort_values(by=['cnt'],ascending=False).reset_index(drop=True)
        tmp['%s_le'%col]=range(len(tmp),0,-1)
        tmp_label=dict(zip(tmp[col],tmp['%s_le'%col]))
        data1[col]=data1[col].map(tmp_label)
    
    time_feat5=['首保','常规保养', '附件安装', '一般维修', '事故车维修', '普通索赔', 
              '服务活动', '二手车认证维修', '内部车辆维修']
    
#     cv_feat=['用户属性','历史修理类型']
    cv_feat=['用户属性']
    train_cols1=train_cols1+cv_feat
    for col in cv_feat:
        tmp=data1.groupby(col)['VIN'].nunique().reset_index(name='cnt')
        tmp=tmp.sort_values(by=['cnt'],ascending=False).reset_index(drop=True)
        print(tmp.head())
        maxx=len(tmp)
        tmp['%s_le'%col]=range(maxx,0,-1)
        tmp['%s_le'%col]=tmp['%s_le'%col].apply(lambda x:x if x>(maxx-100) else (maxx-100) )
        print(tmp['%s_le'%col].nunique())
        tmp_label=dict(zip(tmp[col],tmp['%s_le'%col]))
        data1[col]=data1[col].map(tmp_label)
        
    label_feat=['天数差_next', '公里数差_next', '日均_next']
    
    return data1,train_cols1,time_feat1,time_feat2,time_feat3,time_feat4,time_feat5,label_feat

In [9]:
def get_time_kuanbiao(data1,the_baoyang_id,time_feat1,time_feat2,time_feat3,time_feat4,time_feat5,baoyang_id_max):
    
    if the_baoyang_id<=9: #因为get_time_feat放到了9
#         print("如果baoyang_batch<=9,那么baoyang_id按照时间顺序排序")
        check=data1[['VIN','baoyang_id']+time_feat1+time_feat2+time_feat3+time_feat4+time_feat5].drop_duplicates()
        check['baoyang_id']=check['baoyang_id'].astype(int).astype(str)
        checkt=pd.pivot(check,index='VIN',columns='baoyang_id')
        checkt.columns=['_'.join(list(checkt.columns[x])) for x in range(len(checkt.columns))] 
        checkt=checkt.reset_index()
#         print('宽表2 数据检查')
#         print(checkt.shape)
#         print(checkt['VIN'].nunique())
#         print(checkt.head())
        
    elif (the_baoyang_id>9) and (the_baoyang_id<baoyang_id_max):
#         print('batch的训练模型同上,但是baoyang_id取倒序')
        if 'baoyang_id' in list(data1.columns):
            del data1['baoyang_id']
        data1['baoyang_id']=data1.groupby(['VIN'])['修理日期'].rank(method='dense',ascending=False)
        data11=data1[data1['baoyang_id']<=8]
        check=data11[['VIN','baoyang_id']+time_feat1+time_feat2+time_feat3+time_feat4+time_feat5].drop_duplicates()
        check['baoyang_id']=check['baoyang_id'].astype(int).astype(str)
        checkt=pd.pivot(check,index='VIN',columns='baoyang_id')
        checkt.columns=['_'.join(list(checkt.columns[x])) for x in range(len(checkt.columns))] 
        checkt=checkt.reset_index()
#         print('宽表2 数据检查')
#         print(checkt.shape)
#         print(checkt['VIN'].nunique())
#         print(checkt.head())
    
    elif the_baoyang_id>=baoyang_id_max:
#         print('并且其中baoyang_id是倒序,取最近8次数据')
        if 'baoyang_id' in list(data1.columns):
            del data1['baoyang_id']
        data1['baoyang_id']=data1.groupby(['VIN'])['修理日期'].rank(method='dense',ascending=False)
        data11=data1[data1['baoyang_id']<=8]
        check=data11[['VIN','baoyang_id']+time_feat1+time_feat2+time_feat3+time_feat4+time_feat5].drop_duplicates()
        check['baoyang_id']=check['baoyang_id'].astype(int).astype(str)
        checkt=pd.pivot(check,index='VIN',columns='baoyang_id')
        checkt.columns=['_'.join(list(checkt.columns[x])) for x in range(len(checkt.columns))] 
        checkt=checkt.reset_index()
#         print('宽表2 数据检查')
#         print(checkt.shape)
#         print(checkt['VIN'].nunique)
#         print(checkt.head())
    return checkt

In [10]:
def get_time_feat_235(train1):
    time_feat2=['工时费', '维修金额', '零件费'] 
    time_feat3=['是否有折扣','year', 'mon','xun', 'weekdays', 'is_workday']
    time_feat5=['首保','常规保养', '附件安装', '一般维修', '事故车维修', '普通索赔', 
              '服务活动', '二手车认证维修', '内部车辆维修']

    checkt_train=train1.groupby('VIN')[time_feat2].agg({'mean','sum'})
    checkt_train.columns=['_'.join(x) for x in checkt_train.columns.tolist()]
    checkt_train=checkt_train.reset_index()

    tmp=train1.groupby('VIN')[time_feat3+time_feat5].sum()
    tmp.columns=['%s_sum'%x for x in tmp.columns.tolist()]
    tmp=tmp.reset_index()
    
    tmpp=train1.groupby('VIN')['修理日期'].nunique().reset_index(name='总进店次数')
    tmp=tmp.merge(tmpp,on=['VIN'],how='left')
    for col in time_feat3+time_feat5:
        tmp['%s_ratio'%col]=tmp['%s_sum'%col]/tmp['总进店次数']
    checkt_train=checkt_train.merge(tmp,on=['VIN'],how='left')
    
#     print(checkt_train.shape)
#     print(checkt_train['VIN'].nunique())
    return checkt_train

In [11]:
def get_one_batch_data(dt,the_baoyang_id,baoyang_id_max):
    train_cols2=[]

    data1=get_right_data(dt,the_baoyang_id,baoyang_id_max)
    
    data1,train_cols1,time_feat1,time_feat2,time_feat3,time_feat4,time_feat5,label_feat=get_the_dt_onehot(data1,the_baoyang_id,baoyang_id_max)

    before_cols=list(data1.columns)
    
    imp_checkt1=data1[['VIN','购车时间','在店状态','保养状态','人群_一级分类','人群_二级分类']].drop_duplicates().reset_index(drop=True)
        
    if the_baoyang_id<=9:
        other=data1[['VIN']+train_cols1+label_feat].drop_duplicates().reset_index(drop=True)
        print('宽表1 数据检查')
        print(other.shape)
        print(other['VIN'].nunique())
        
        checkt=get_time_kuanbiao(data1,the_baoyang_id,time_feat1,time_feat2,time_feat3,time_feat4,time_feat5,baoyang_id_max)
        print('宽表2 数据检查')
        print(checkt.shape)
        print(checkt['VIN'].nunique())
        checkt=checkt.merge(other,on=['VIN'],how='left')
        
        if the_baoyang_id==2:
            checkt=get_time_feat2(checkt,time_feat1,the_baoyang_id) #顺序
        elif the_baoyang_id==3:
            checkt=get_time_feat3(checkt,time_feat1,the_baoyang_id)#顺序
        elif the_baoyang_id==4:
            checkt=get_time_feat4(checkt,time_feat1,the_baoyang_id)#顺序
        elif the_baoyang_id==5:
            checkt=get_time_feat5(checkt,time_feat1,the_baoyang_id)#顺序
        elif the_baoyang_id==6:
            checkt=get_time_feat6(checkt,time_feat1,the_baoyang_id)#顺序
        elif the_baoyang_id==7:
            checkt=get_time_feat7(checkt,time_feat1,the_baoyang_id)#顺序
        elif the_baoyang_id==8:
            checkt=get_time_feat8(checkt,time_feat1,the_baoyang_id)#顺序
        elif the_baoyang_id==9: 
            checkt=get_time_feat9(checkt,time_feat1,the_baoyang_id) #顺序
            
        print('宽表3 数据检查')
        print(checkt.shape)
        print(checkt['VIN'].nunique())
        
        end_id=the_baoyang_id-1

        imp_checkt2=checkt[['VIN','天数_%d'%end_id,'公里数_%d'%end_id]].drop_duplicates().reset_index(drop=True)
        imp_checkt2.columns=['VIN','期间最后一次保养天数','期间最后一次保养公里数']
        
        train_cols2=[x for x in list(checkt.columns) if x not in before_cols]
        checkt=checkt.merge(imp_checkt1,on=['VIN'],how='left').merge(imp_checkt2,on=['VIN'],how='left')
        
    elif the_baoyang_id==baoyang_id_max:

        train1=data1[data1['数据集标注']=='训练集']
        print('train数据集')
        print(train1.shape)
        print(train1['VIN'].nunique())
        other_train=train1[['VIN']+train_cols1+label_feat].drop_duplicates().reset_index(drop=True)
        print(other_train.shape)
        print(other_train['VIN'].nunique())

        test1=data1[data1['数据集标注']=='测试集']
        print('test数据集')
        print(test1.shape)
        print(test1['VIN'].nunique())
        other_test=test1[['VIN']+train_cols1+label_feat].drop_duplicates().reset_index(drop=True)
        print(other_test.shape)
        print(other_test['VIN'].nunique())
        
        print(test1.shape[0]-train1.shape[0]==test1['VIN'].nunique())

        checkt_train=get_time_feat_235(train1)
        print('train and test数据,经过get_time_feat_235以后的数据检查')
        print(checkt_train.shape)
        print(checkt_train['VIN'].nunique())
        checkt_test=get_time_feat_235(test1)
        print(checkt_test.shape)
        print(checkt_test['VIN'].nunique())
                
        checkt_train1=get_time_kuanbiao(train1,the_baoyang_id,time_feat1,time_feat2,time_feat3,time_feat4,time_feat5,baoyang_id_max) #放了最近8次的纪录
        print('经过get_time_kuanbiao之后的检查:')
        print(checkt_train1.shape)
        print(checkt_train1['VIN'].nunique())
        checkt_test1=get_time_kuanbiao(test1,the_baoyang_id,time_feat1,time_feat2,time_feat3,time_feat4,time_feat5,baoyang_id_max) #放了最近8次的纪录
        print(checkt_test1.shape)
        print(checkt_test1['VIN'].nunique())
        
        checkt_train_fin=checkt_train.merge(other_train,on=['VIN'],how='left').merge(checkt_train1,on=['VIN'],how='left')
        checkt_train_fin['数据集标注']='训练集'
        checkt_test_fin=checkt_test.merge(other_test,on=['VIN'],how='left').merge(checkt_test1,on=['VIN'],how='left')
        checkt_test_fin['数据集标注']='测试集'
        
        print("train 宽表3 检查")
        print(checkt_train_fin.shape)
        print(checkt_train_fin['VIN'].nunique())
        print(pd.isnull(checkt_train_fin[['VIN','天数差_next','公里数差_next']].drop_duplicates()).sum())
        
        print("test 宽表3 检查")
        print(checkt_test_fin.shape)
        print(checkt_test_fin['VIN'].nunique())   
        print(pd.isnull(checkt_test_fin[['VIN','天数差_next','公里数差_next']].drop_duplicates()).sum())
        
        print('将train and test合并,得到最后一个batch的宽表')
        checkt=pd.concat([checkt_train_fin,checkt_test_fin],axis=0)
        checkt=get_time_feat_above_9(checkt,time_feat1)
        
        imp_checkt2=checkt[['VIN','数据集标注','天数_1','公里数_1']].drop_duplicates().reset_index(drop=True)
        imp_checkt2.columns=['VIN','数据集标注','期间最后一次保养天数','期间最后一次保养公里数']
        
        train_cols2=[x for x in list(checkt.columns) if x not in before_cols]
        checkt=checkt.merge(imp_checkt1,on=['VIN'],how='left').merge(imp_checkt2,on=['VIN','数据集标注'],how='left')
 
    elif (the_baoyang_id>9) & (the_baoyang_id<baoyang_id_max):
        other=data1[['VIN']+train_cols1+label_feat].drop_duplicates().reset_index(drop=True)
        print('宽表1 数据检查')
        print(other.shape)
        print(other['VIN'].nunique())
        
        checkt=get_time_kuanbiao(data1,the_baoyang_id,time_feat1,time_feat2,time_feat3,time_feat4,time_feat5,baoyang_id_max)
        checkt=checkt.merge(other,on=['VIN'],how='left')
        
        checkt=get_time_feat_above_9(checkt,time_feat1) #倒序!!!保留其最近8次数据即可
        print('宽表3 数据检查')
        print(checkt.shape)
        print(checkt['VIN'].nunique())
        
        imp_checkt2=checkt[['VIN','天数_1','公里数_1']].drop_duplicates().reset_index(drop=True)
        imp_checkt2.columns=['VIN','期间最后一次保养天数','期间最后一次保养公里数']
        
        train_cols2=[x for x in list(checkt.columns) if x not in before_cols]
        checkt=checkt.merge(imp_checkt1,on=['VIN'],how='left').merge(imp_checkt2,on=['VIN'],how='left')

    return checkt,train_cols1,train_cols2

In [12]:
def get_time_feat_above_9(checkt,time_feat1):
    
    end_id=8
    
    for col in ['天数','公里数','日均']:
        
        checkt['%s_1_2'%col]=checkt['%s_1'%col]/checkt['%s_2'%col]
        checkt['%s_1_3'%col]=checkt['%s_1'%col]/checkt['%s_3'%col]
        checkt['%s_1_4'%col]=checkt['%s_1'%col]/checkt['%s_4'%col]
        checkt['%s_1_5'%col]=checkt['%s_1'%col]/checkt['%s_5'%col]
        checkt['%s_1_6'%col]=checkt['%s_1'%col]/checkt['%s_6'%col]
        checkt['%s_1_7'%col]=checkt['%s_1'%col]/checkt['%s_7'%col]
        checkt['%s_1_8'%col]=checkt['%s_1'%col]/checkt['%s_8'%col]
        
        checkt['%s_2_3'%col]=checkt['%s_2'%col]/checkt['%s_3'%col]
        checkt['%s_2_4'%col]=checkt['%s_2'%col]/checkt['%s_4'%col]
        checkt['%s_2_5'%col]=checkt['%s_2'%col]/checkt['%s_5'%col]
        checkt['%s_2_6'%col]=checkt['%s_2'%col]/checkt['%s_6'%col]
        checkt['%s_2_7'%col]=checkt['%s_2'%col]/checkt['%s_7'%col]
        checkt['%s_2_8'%col]=checkt['%s_2'%col]/checkt['%s_8'%col]
        
        checkt['%s_3_4'%col]=checkt['%s_3'%col]/checkt['%s_4'%col]
        checkt['%s_3_5'%col]=checkt['%s_3'%col]/checkt['%s_5'%col]
        checkt['%s_3_6'%col]=checkt['%s_3'%col]/checkt['%s_6'%col]
        checkt['%s_3_7'%col]=checkt['%s_3'%col]/checkt['%s_7'%col]
        checkt['%s_3_8'%col]=checkt['%s_3'%col]/checkt['%s_8'%col]
        
        checkt['%s_4_5'%col]=checkt['%s_4'%col]/checkt['%s_5'%col]
        checkt['%s_4_6'%col]=checkt['%s_4'%col]/checkt['%s_6'%col]
        checkt['%s_4_7'%col]=checkt['%s_4'%col]/checkt['%s_7'%col]
        checkt['%s_4_8'%col]=checkt['%s_4'%col]/checkt['%s_8'%col]
        
        checkt['%s_5_6'%col]=checkt['%s_5'%col]/checkt['%s_6'%col]
        checkt['%s_5_7'%col]=checkt['%s_5'%col]/checkt['%s_7'%col]
        checkt['%s_5_8'%col]=checkt['%s_5'%col]/checkt['%s_8'%col]
        
        checkt['%s_6_7'%col]=checkt['%s_6'%col]/checkt['%s_7'%col]
        checkt['%s_6_8'%col]=checkt['%s_6'%col]/checkt['%s_8'%col]
        
        checkt['%s_7_8'%col]=checkt['%s_7'%col]/checkt['%s_8'%col]
    
    for col in ['天数差','公里数差','日均差']:
        checkt['%s_mean'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].mean(1)
        checkt['%s_med'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].median(1)
        checkt['%s_min'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].min(1)
        checkt['%s_max'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].max(1)
        
        checkt['%s_1_2_diff'%col]=checkt['%s_1'%col]-checkt['%s_2'%col]
        checkt['%s_1_3_diff'%col]=checkt['%s_1'%col]-checkt['%s_3'%col]
        checkt['%s_1_4_diff'%col]=checkt['%s_1'%col]-checkt['%s_4'%col]
        checkt['%s_1_5_diff'%col]=checkt['%s_1'%col]-checkt['%s_5'%col]
        checkt['%s_1_6_diff'%col]=checkt['%s_1'%col]-checkt['%s_6'%col]
        checkt['%s_1_7_diff'%col]=checkt['%s_1'%col]-checkt['%s_7'%col]
        checkt['%s_1_8_diff'%col]=checkt['%s_1'%col]-checkt['%s_8'%col]
    
        checkt['%s_2_3_diff'%col]=checkt['%s_2'%col]-checkt['%s_3'%col]
        checkt['%s_2_4_diff'%col]=checkt['%s_2'%col]-checkt['%s_4'%col]
        checkt['%s_2_5_diff'%col]=checkt['%s_2'%col]-checkt['%s_5'%col]
        checkt['%s_2_6_diff'%col]=checkt['%s_2'%col]-checkt['%s_6'%col]
        checkt['%s_2_7_diff'%col]=checkt['%s_2'%col]-checkt['%s_7'%col]
        checkt['%s_2_8_diff'%col]=checkt['%s_2'%col]-checkt['%s_8'%col]
        
        checkt['%s_3_4_diff'%col]=checkt['%s_3'%col]-checkt['%s_4'%col]
        checkt['%s_3_5_diff'%col]=checkt['%s_3'%col]-checkt['%s_5'%col]
        checkt['%s_3_6_diff'%col]=checkt['%s_3'%col]-checkt['%s_6'%col]
        checkt['%s_3_7_diff'%col]=checkt['%s_3'%col]-checkt['%s_7'%col]
        checkt['%s_3_8_diff'%col]=checkt['%s_3'%col]-checkt['%s_8'%col]
        
        checkt['%s_4_5_diff'%col]=checkt['%s_4'%col]-checkt['%s_5'%col]
        checkt['%s_4_6_diff'%col]=checkt['%s_4'%col]-checkt['%s_6'%col]
        checkt['%s_4_7_diff'%col]=checkt['%s_4'%col]-checkt['%s_7'%col]
        checkt['%s_4_8_diff'%col]=checkt['%s_4'%col]-checkt['%s_8'%col]
        
        checkt['%s_5_6_diff'%col]=checkt['%s_5'%col]-checkt['%s_6'%col]
        checkt['%s_5_7_diff'%col]=checkt['%s_5'%col]-checkt['%s_7'%col]
        checkt['%s_5_8_diff'%col]=checkt['%s_5'%col]-checkt['%s_8'%col]
        
        checkt['%s_6_7_diff'%col]=checkt['%s_6'%col]-checkt['%s_7'%col]
        checkt['%s_6_8_diff'%col]=checkt['%s_6'%col]-checkt['%s_8'%col]
        
        checkt['%s_7_8_diff'%col]=checkt['%s_7'%col]-checkt['%s_8'%col]
        
        checkt['%s_1_2'%col]=checkt['%s_1'%col]/checkt['%s_2'%col]
        checkt['%s_1_3'%col]=checkt['%s_1'%col]/checkt['%s_3'%col]
        checkt['%s_1_4'%col]=checkt['%s_1'%col]/checkt['%s_4'%col]
        checkt['%s_1_5'%col]=checkt['%s_1'%col]/checkt['%s_5'%col]
        checkt['%s_1_6'%col]=checkt['%s_1'%col]/checkt['%s_6'%col]
        checkt['%s_1_7'%col]=checkt['%s_1'%col]/checkt['%s_7'%col]
        checkt['%s_1_8'%col]=checkt['%s_1'%col]/checkt['%s_8'%col]
        
        checkt['%s_2_3'%col]=checkt['%s_2'%col]/checkt['%s_3'%col]
        checkt['%s_2_4'%col]=checkt['%s_2'%col]/checkt['%s_4'%col]
        checkt['%s_2_5'%col]=checkt['%s_2'%col]/checkt['%s_5'%col]
        checkt['%s_2_6'%col]=checkt['%s_2'%col]/checkt['%s_6'%col]
        checkt['%s_2_7'%col]=checkt['%s_2'%col]/checkt['%s_7'%col]
        checkt['%s_2_8'%col]=checkt['%s_2'%col]/checkt['%s_8'%col]
        
        checkt['%s_3_4'%col]=checkt['%s_3'%col]/checkt['%s_4'%col]
        checkt['%s_3_5'%col]=checkt['%s_3'%col]/checkt['%s_5'%col]
        checkt['%s_3_6'%col]=checkt['%s_3'%col]/checkt['%s_6'%col]
        checkt['%s_3_7'%col]=checkt['%s_3'%col]/checkt['%s_7'%col]
        checkt['%s_3_8'%col]=checkt['%s_3'%col]/checkt['%s_8'%col]
        
        checkt['%s_4_5'%col]=checkt['%s_4'%col]/checkt['%s_5'%col]
        checkt['%s_4_6'%col]=checkt['%s_4'%col]/checkt['%s_6'%col]
        checkt['%s_4_7'%col]=checkt['%s_4'%col]/checkt['%s_7'%col]
        checkt['%s_4_8'%col]=checkt['%s_4'%col]/checkt['%s_8'%col]
        
        checkt['%s_5_6'%col]=checkt['%s_5'%col]/checkt['%s_6'%col]
        checkt['%s_5_7'%col]=checkt['%s_5'%col]/checkt['%s_7'%col]
        checkt['%s_5_8'%col]=checkt['%s_5'%col]/checkt['%s_8'%col]
        
        checkt['%s_6_7'%col]=checkt['%s_6'%col]/checkt['%s_7'%col]
        checkt['%s_6_8'%col]=checkt['%s_6'%col]/checkt['%s_8'%col]
        
        checkt['%s_7_8'%col]=checkt['%s_7'%col]/checkt['%s_8'%col]
        
        checkt['%s_1_2_inc'%col]=checkt['%s_1_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_1_3_inc'%col]=checkt['%s_1_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_1_4_inc'%col]=checkt['%s_1_4_diff'%col]/checkt['%s_4'%col]
        checkt['%s_1_5_inc'%col]=checkt['%s_1_5_diff'%col]/checkt['%s_5'%col]
        checkt['%s_1_6_inc'%col]=checkt['%s_1_6_diff'%col]/checkt['%s_6'%col]
        checkt['%s_1_7_inc'%col]=checkt['%s_1_7_diff'%col]/checkt['%s_7'%col]
        checkt['%s_1_8_inc'%col]=checkt['%s_1_8_diff'%col]/checkt['%s_8'%col]
        
        checkt['%s_2_3_inc'%col]=checkt['%s_2_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_2_4_inc'%col]=checkt['%s_2_4_diff'%col]/checkt['%s_4'%col]
        checkt['%s_2_5_inc'%col]=checkt['%s_2_5_diff'%col]/checkt['%s_5'%col]
        checkt['%s_2_6_inc'%col]=checkt['%s_2_6_diff'%col]/checkt['%s_6'%col]
        checkt['%s_2_7_inc'%col]=checkt['%s_2_7_diff'%col]/checkt['%s_7'%col]
        checkt['%s_2_8_inc'%col]=checkt['%s_2_8_diff'%col]/checkt['%s_8'%col]
        
        checkt['%s_3_4_inc'%col]=checkt['%s_3_4_diff'%col]/checkt['%s_4'%col]
        checkt['%s_3_5_inc'%col]=checkt['%s_3_5_diff'%col]/checkt['%s_5'%col]
        checkt['%s_3_6_inc'%col]=checkt['%s_3_6_diff'%col]/checkt['%s_6'%col]
        checkt['%s_3_7_inc'%col]=checkt['%s_3_7_diff'%col]/checkt['%s_7'%col]
        checkt['%s_3_8_inc'%col]=checkt['%s_3_8_diff'%col]/checkt['%s_8'%col]
        
        checkt['%s_4_5_inc'%col]=checkt['%s_4_5_diff'%col]/checkt['%s_5'%col]
        checkt['%s_4_6_inc'%col]=checkt['%s_4_6_diff'%col]/checkt['%s_6'%col]
        checkt['%s_4_7_inc'%col]=checkt['%s_4_7_diff'%col]/checkt['%s_7'%col]
        checkt['%s_4_8_inc'%col]=checkt['%s_4_8_diff'%col]/checkt['%s_8'%col]
        
        checkt['%s_5_6_inc'%col]=checkt['%s_5_6_diff'%col]/checkt['%s_6'%col]
        checkt['%s_5_7_inc'%col]=checkt['%s_5_7_diff'%col]/checkt['%s_7'%col]
        checkt['%s_5_8_inc'%col]=checkt['%s_5_8_diff'%col]/checkt['%s_8'%col]
        
        checkt['%s_6_7_inc'%col]=checkt['%s_6_7_diff'%col]/checkt['%s_7'%col]
        checkt['%s_6_8_inc'%col]=checkt['%s_6_8_diff'%col]/checkt['%s_8'%col]
        
        checkt['%s_7_8_inc'%col]=checkt['%s_7_8_diff'%col]/checkt['%s_8'%col]

    for col in ['天数','公里数','日均']:

        checkt['%s比_7_8'%col]=checkt['%s_7'%col]/checkt['%s差_8'%col]

        checkt['%s比_6_8'%col]=checkt['%s_6'%col]/checkt['%s差_8'%col]
        checkt['%s比_6_7'%col]=checkt['%s_6'%col]/checkt['%s差_7'%col]
    
        checkt['%s比_5_8'%col]=checkt['%s_5'%col]/checkt['%s差_8'%col]
        checkt['%s比_5_7'%col]=checkt['%s_5'%col]/checkt['%s差_7'%col]
        checkt['%s比_5_6'%col]=checkt['%s_5'%col]/checkt['%s差_6'%col]
    
        checkt['%s比_4_8'%col]=checkt['%s_4'%col]/checkt['%s差_8'%col]
        checkt['%s比_4_7'%col]=checkt['%s_4'%col]/checkt['%s差_7'%col]
        checkt['%s比_4_6'%col]=checkt['%s_4'%col]/checkt['%s差_6'%col]
        checkt['%s比_4_5'%col]=checkt['%s_4'%col]/checkt['%s差_5'%col]
    
        checkt['%s比_3_8'%col]=checkt['%s_3'%col]/checkt['%s差_8'%col]
        checkt['%s比_3_7'%col]=checkt['%s_3'%col]/checkt['%s差_7'%col]
        checkt['%s比_3_6'%col]=checkt['%s_3'%col]/checkt['%s差_6'%col]
        checkt['%s比_3_5'%col]=checkt['%s_3'%col]/checkt['%s差_5'%col]
        checkt['%s比_3_4'%col]=checkt['%s_3'%col]/checkt['%s差_4'%col]
    
        checkt['%s比_2_8'%col]=checkt['%s_2'%col]/checkt['%s差_8'%col]
        checkt['%s比_2_7'%col]=checkt['%s_2'%col]/checkt['%s差_7'%col]
        checkt['%s比_2_6'%col]=checkt['%s_2'%col]/checkt['%s差_6'%col]
        checkt['%s比_2_5'%col]=checkt['%s_2'%col]/checkt['%s差_5'%col]
        checkt['%s比_2_4'%col]=checkt['%s_2'%col]/checkt['%s差_4'%col]
        checkt['%s比_2_3'%col]=checkt['%s_2'%col]/checkt['%s差_3'%col]
        
        checkt['%s比_1_8'%col]=checkt['%s_1'%col]/checkt['%s差_8'%col]
        checkt['%s比_1_7'%col]=checkt['%s_1'%col]/checkt['%s差_7'%col]
        checkt['%s比_1_6'%col]=checkt['%s_1'%col]/checkt['%s差_6'%col]
        checkt['%s比_1_5'%col]=checkt['%s_1'%col]/checkt['%s差_5'%col]
        checkt['%s比_1_4'%col]=checkt['%s_1'%col]/checkt['%s差_4'%col]
        checkt['%s比_1_3'%col]=checkt['%s_1'%col]/checkt['%s差_3'%col]
        checkt['%s比_1_2'%col]=checkt['%s_1'%col]/checkt['%s差_2'%col]
    
#     print('thebaoyang id在9次以上的batch的数据检查')
#     print(checkt.shape)
#     print(checkt['VIN'].nunique())
    return checkt


# In[171]:


def get_time_feat9(checkt,time_feat1,the_baoyang_id):
    end_id=the_baoyang_id-1

    for col in ['天数','公里数','日均']:
        checkt['%s_8_7'%col]=checkt['%s_8'%col]/checkt['%s_7'%col]
        checkt['%s_8_6'%col]=checkt['%s_8'%col]/checkt['%s_6'%col]
        checkt['%s_8_5'%col]=checkt['%s_8'%col]/checkt['%s_5'%col]
        checkt['%s_8_4'%col]=checkt['%s_8'%col]/checkt['%s_4'%col]
        checkt['%s_8_3'%col]=checkt['%s_8'%col]/checkt['%s_3'%col]
        checkt['%s_8_2'%col]=checkt['%s_8'%col]/checkt['%s_2'%col]
        checkt['%s_8_1'%col]=checkt['%s_8'%col]/checkt['%s_1'%col]
        checkt['%s_7_6'%col]=checkt['%s_7'%col]/checkt['%s_6'%col]
        checkt['%s_7_5'%col]=checkt['%s_7'%col]/checkt['%s_5'%col]
        checkt['%s_7_4'%col]=checkt['%s_7'%col]/checkt['%s_4'%col]
        checkt['%s_7_3'%col]=checkt['%s_7'%col]/checkt['%s_3'%col]
        checkt['%s_7_2'%col]=checkt['%s_7'%col]/checkt['%s_2'%col]
        checkt['%s_7_1'%col]=checkt['%s_7'%col]/checkt['%s_1'%col]
        checkt['%s_6_5'%col]=checkt['%s_6'%col]/checkt['%s_5'%col]
        checkt['%s_6_4'%col]=checkt['%s_6'%col]/checkt['%s_4'%col]
        checkt['%s_6_3'%col]=checkt['%s_6'%col]/checkt['%s_3'%col]
        checkt['%s_6_2'%col]=checkt['%s_6'%col]/checkt['%s_2'%col]
        checkt['%s_6_1'%col]=checkt['%s_6'%col]/checkt['%s_1'%col]
        checkt['%s_5_4'%col]=checkt['%s_5'%col]/checkt['%s_4'%col]
        checkt['%s_5_3'%col]=checkt['%s_5'%col]/checkt['%s_3'%col]
        checkt['%s_5_2'%col]=checkt['%s_5'%col]/checkt['%s_2'%col]
        checkt['%s_5_1'%col]=checkt['%s_5'%col]/checkt['%s_1'%col]
        checkt['%s_4_3'%col]=checkt['%s_4'%col]/checkt['%s_3'%col]
        checkt['%s_4_2'%col]=checkt['%s_4'%col]/checkt['%s_2'%col]
        checkt['%s_4_1'%col]=checkt['%s_4'%col]/checkt['%s_1'%col]
        checkt['%s_3_2'%col]=checkt['%s_3'%col]/checkt['%s_2'%col]
        checkt['%s_3_1'%col]=checkt['%s_3'%col]/checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]

    for col in ['天数差','公里数差','日均差']:
        checkt['%s_mean'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].mean(1)
        checkt['%s_med'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].median(1)
        checkt['%s_min'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].min(1)
        checkt['%s_max'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].max(1)
        
        checkt['%s_8_7_diff'%col]=checkt['%s_8'%col]-checkt['%s_7'%col]
        checkt['%s_8_6_diff'%col]=checkt['%s_8'%col]-checkt['%s_6'%col]
        checkt['%s_8_5_diff'%col]=checkt['%s_8'%col]-checkt['%s_5'%col]
        checkt['%s_8_4_diff'%col]=checkt['%s_8'%col]-checkt['%s_4'%col]
        checkt['%s_8_3_diff'%col]=checkt['%s_8'%col]-checkt['%s_3'%col]
        checkt['%s_8_2_diff'%col]=checkt['%s_8'%col]-checkt['%s_2'%col]
        checkt['%s_8_1_diff'%col]=checkt['%s_8'%col]-checkt['%s_1'%col]
        checkt['%s_7_6_diff'%col]=checkt['%s_7'%col]-checkt['%s_6'%col]
        checkt['%s_7_5_diff'%col]=checkt['%s_7'%col]-checkt['%s_5'%col]
        checkt['%s_7_4_diff'%col]=checkt['%s_7'%col]-checkt['%s_4'%col]
        checkt['%s_7_3_diff'%col]=checkt['%s_7'%col]-checkt['%s_3'%col]
        checkt['%s_7_2_diff'%col]=checkt['%s_7'%col]-checkt['%s_2'%col]
        checkt['%s_7_1_diff'%col]=checkt['%s_7'%col]-checkt['%s_1'%col]
        checkt['%s_6_5_diff'%col]=checkt['%s_6'%col]-checkt['%s_5'%col]
        checkt['%s_6_4_diff'%col]=checkt['%s_6'%col]-checkt['%s_4'%col]
        checkt['%s_6_3_diff'%col]=checkt['%s_6'%col]-checkt['%s_3'%col]
        checkt['%s_6_2_diff'%col]=checkt['%s_6'%col]-checkt['%s_2'%col]
        checkt['%s_6_1_diff'%col]=checkt['%s_6'%col]-checkt['%s_1'%col]
        checkt['%s_5_4_diff'%col]=checkt['%s_5'%col]-checkt['%s_4'%col]
        checkt['%s_5_3_diff'%col]=checkt['%s_5'%col]-checkt['%s_3'%col]
        checkt['%s_5_2_diff'%col]=checkt['%s_5'%col]-checkt['%s_2'%col]
        checkt['%s_5_1_diff'%col]=checkt['%s_5'%col]-checkt['%s_1'%col]
        checkt['%s_4_3_diff'%col]=checkt['%s_4'%col]-checkt['%s_3'%col]
        checkt['%s_4_2_diff'%col]=checkt['%s_4'%col]-checkt['%s_2'%col]
        checkt['%s_4_1_diff'%col]=checkt['%s_4'%col]-checkt['%s_1'%col]
        checkt['%s_3_2_diff'%col]=checkt['%s_3'%col]-checkt['%s_2'%col]
        checkt['%s_3_1_diff'%col]=checkt['%s_3'%col]-checkt['%s_1'%col]
        checkt['%s_2_1_diff'%col]=checkt['%s_2'%col]-checkt['%s_1'%col]
        
        checkt['%s_8_7'%col]=checkt['%s_8'%col]/checkt['%s_7'%col]
        checkt['%s_8_6'%col]=checkt['%s_8'%col]/checkt['%s_6'%col]
        checkt['%s_8_5'%col]=checkt['%s_8'%col]/checkt['%s_5'%col]
        checkt['%s_8_4'%col]=checkt['%s_8'%col]/checkt['%s_4'%col]
        checkt['%s_8_3'%col]=checkt['%s_8'%col]/checkt['%s_3'%col]
        checkt['%s_8_2'%col]=checkt['%s_8'%col]/checkt['%s_2'%col]
        checkt['%s_8_1'%col]=checkt['%s_8'%col]/checkt['%s_1'%col]
        checkt['%s_7_6'%col]=checkt['%s_7'%col]/checkt['%s_6'%col]
        checkt['%s_7_5'%col]=checkt['%s_7'%col]/checkt['%s_5'%col]
        checkt['%s_7_4'%col]=checkt['%s_7'%col]/checkt['%s_4'%col]
        checkt['%s_7_3'%col]=checkt['%s_7'%col]/checkt['%s_3'%col]
        checkt['%s_7_2'%col]=checkt['%s_7'%col]/checkt['%s_2'%col]
        checkt['%s_7_1'%col]=checkt['%s_7'%col]/checkt['%s_1'%col]
        checkt['%s_6_5'%col]=checkt['%s_6'%col]/checkt['%s_5'%col]
        checkt['%s_6_4'%col]=checkt['%s_6'%col]/checkt['%s_4'%col]
        checkt['%s_6_3'%col]=checkt['%s_6'%col]/checkt['%s_3'%col]
        checkt['%s_6_2'%col]=checkt['%s_6'%col]/checkt['%s_2'%col]
        checkt['%s_6_1'%col]=checkt['%s_6'%col]/checkt['%s_1'%col]
        checkt['%s_5_4'%col]=checkt['%s_5'%col]/checkt['%s_4'%col]
        checkt['%s_5_3'%col]=checkt['%s_5'%col]/checkt['%s_3'%col]
        checkt['%s_5_3'%col]=checkt['%s_5'%col]/checkt['%s_2'%col]
        checkt['%s_5_1'%col]=checkt['%s_5'%col]/checkt['%s_1'%col]
        checkt['%s_4_3'%col]=checkt['%s_4'%col]/checkt['%s_3'%col]
        checkt['%s_4_2'%col]=checkt['%s_4'%col]/checkt['%s_2'%col]
        checkt['%s_4_1'%col]=checkt['%s_4'%col]/checkt['%s_1'%col]
        checkt['%s_3_2'%col]=checkt['%s_3'%col]/checkt['%s_2'%col]
        checkt['%s_3_1'%col]=checkt['%s_3'%col]/checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]
        
        checkt['%s_8_7_inc'%col]=checkt['%s_8_7_diff'%col]/checkt['%s_7'%col]
        checkt['%s_8_6_inc'%col]=checkt['%s_8_6_diff'%col]/checkt['%s_6'%col]
        checkt['%s_8_5_inc'%col]=checkt['%s_8_5_diff'%col]/checkt['%s_5'%col]
        checkt['%s_8_4_inc'%col]=checkt['%s_8_4_diff'%col]/checkt['%s_4'%col]
        checkt['%s_8_3_inc'%col]=checkt['%s_8_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_8_2_inc'%col]=checkt['%s_8_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_7_6_inc'%col]=checkt['%s_7_6_diff'%col]/checkt['%s_6'%col]
        checkt['%s_7_5_inc'%col]=checkt['%s_7_5_diff'%col]/checkt['%s_5'%col]
        checkt['%s_7_4_inc'%col]=checkt['%s_7_4_diff'%col]/checkt['%s_4'%col]
        checkt['%s_7_3_inc'%col]=checkt['%s_7_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_7_2_inc'%col]=checkt['%s_7_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_7_1_inc'%col]=checkt['%s_7_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_6_5_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_5'%col]
        checkt['%s_6_4_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_4'%col]
        checkt['%s_6_3_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_3'%col]
        checkt['%s_6_2_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_2'%col]
        checkt['%s_6_1_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_1'%col]
        checkt['%s_5_4_inc'%col]=checkt['%s_5_4_diff'%col]/checkt['%s_4'%col]
        checkt['%s_5_3_inc'%col]=checkt['%s_5_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_5_2_inc'%col]=checkt['%s_5_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_5_1_inc'%col]=checkt['%s_5_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_4_3_inc'%col]=checkt['%s_4_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_4_2_inc'%col]=checkt['%s_4_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_4_1_inc'%col]=checkt['%s_4_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_3_2_inc'%col]=checkt['%s_3_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_3_1_inc'%col]=checkt['%s_3_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_2_1_inc'%col]=checkt['%s_2_1_diff'%col]/checkt['%s_1'%col]
    
    for col in ['天数','公里数','日均']:
        checkt['%s比_2_2'%col]=checkt['%s_2'%col]/checkt['%s差_2'%col]
        checkt['%s比_2_1'%col]=checkt['%s_2'%col]/checkt['%s差_1'%col]
    
        checkt['%s比_3_1'%col]=checkt['%s_3'%col]/checkt['%s差_1'%col]
        checkt['%s比_3_2'%col]=checkt['%s_3'%col]/checkt['%s差_2'%col]
        checkt['%s比_3_3'%col]=checkt['%s_3'%col]/checkt['%s差_3'%col]
    
        checkt['%s比_4_1'%col]=checkt['%s_4'%col]/checkt['%s差_1'%col]
        checkt['%s比_4_2'%col]=checkt['%s_4'%col]/checkt['%s差_2'%col]
        checkt['%s比_4_3'%col]=checkt['%s_4'%col]/checkt['%s差_3'%col]
        checkt['%s比_4_4'%col]=checkt['%s_4'%col]/checkt['%s差_4'%col]
    
        checkt['%s比_5_5'%col]=checkt['%s_5'%col]/checkt['%s差_5'%col]
        checkt['%s比_5_4'%col]=checkt['%s_5'%col]/checkt['%s差_4'%col]
        checkt['%s比_5_3'%col]=checkt['%s_5'%col]/checkt['%s差_3'%col]
        checkt['%s比_5_2'%col]=checkt['%s_5'%col]/checkt['%s差_2'%col]
        checkt['%s比_5_1'%col]=checkt['%s_5'%col]/checkt['%s差_1'%col]
    
        checkt['%s比_6_6'%col]=checkt['%s_6'%col]/checkt['%s差_6'%col]
        checkt['%s比_6_5'%col]=checkt['%s_6'%col]/checkt['%s差_5'%col]
        checkt['%s比_6_4'%col]=checkt['%s_6'%col]/checkt['%s差_4'%col]
        checkt['%s比_6_3'%col]=checkt['%s_6'%col]/checkt['%s差_3'%col]
        checkt['%s比_6_2'%col]=checkt['%s_6'%col]/checkt['%s差_2'%col]
        checkt['%s比_6_1'%col]=checkt['%s_6'%col]/checkt['%s差_1'%col]
    
        checkt['%s比_7_1'%col]=checkt['%s_7'%col]/checkt['%s差_1'%col]
        checkt['%s比_7_2'%col]=checkt['%s_7'%col]/checkt['%s差_2'%col]
        checkt['%s比_7_3'%col]=checkt['%s_7'%col]/checkt['%s差_3'%col]
        checkt['%s比_7_4'%col]=checkt['%s_7'%col]/checkt['%s差_4'%col]
        checkt['%s比_7_5'%col]=checkt['%s_7'%col]/checkt['%s差_5'%col]
        checkt['%s比_7_6'%col]=checkt['%s_7'%col]/checkt['%s差_6'%col]
        
        checkt['%s比_8_1'%col]=checkt['%s_8'%col]/checkt['%s差_1'%col]
        checkt['%s比_8_2'%col]=checkt['%s_8'%col]/checkt['%s差_2'%col]
        checkt['%s比_8_3'%col]=checkt['%s_8'%col]/checkt['%s差_3'%col]
        checkt['%s比_8_4'%col]=checkt['%s_8'%col]/checkt['%s差_4'%col]
        checkt['%s比_8_5'%col]=checkt['%s_8'%col]/checkt['%s差_5'%col]
        checkt['%s比_8_6'%col]=checkt['%s_8'%col]/checkt['%s差_6'%col]
        checkt['%s比_8_7'%col]=checkt['%s_8'%col]/checkt['%s差_7'%col]

    print(checkt.shape)
    print(checkt['VIN'].nunique())
    return checkt


# In[145]:


def get_time_feat8(checkt,time_feat1,the_baoyang_id):
    end_id=the_baoyang_id-1

#     for i in range(1,the_baoyang_id):
#         checkt['日均_%d'%i]=checkt['公里数_%d'%i]/checkt['天数_%d'%i] #整段日均

    for col in ['天数','公里数','日均']:
        checkt['%s_7_6'%col]=checkt['%s_7'%col]/checkt['%s_6'%col]
        checkt['%s_7_5'%col]=checkt['%s_7'%col]/checkt['%s_5'%col]
        checkt['%s_7_4'%col]=checkt['%s_7'%col]/checkt['%s_4'%col]
        checkt['%s_7_3'%col]=checkt['%s_7'%col]/checkt['%s_3'%col]
        checkt['%s_7_2'%col]=checkt['%s_7'%col]/checkt['%s_2'%col]
        checkt['%s_7_1'%col]=checkt['%s_7'%col]/checkt['%s_1'%col]
        checkt['%s_6_5'%col]=checkt['%s_6'%col]/checkt['%s_5'%col]
        checkt['%s_6_4'%col]=checkt['%s_6'%col]/checkt['%s_4'%col]
        checkt['%s_6_3'%col]=checkt['%s_6'%col]/checkt['%s_3'%col]
        checkt['%s_6_2'%col]=checkt['%s_6'%col]/checkt['%s_2'%col]
        checkt['%s_6_1'%col]=checkt['%s_6'%col]/checkt['%s_1'%col]
        checkt['%s_5_4'%col]=checkt['%s_5'%col]/checkt['%s_4'%col]
        checkt['%s_5_3'%col]=checkt['%s_5'%col]/checkt['%s_3'%col]
        checkt['%s_5_2'%col]=checkt['%s_5'%col]/checkt['%s_2'%col]
        checkt['%s_5_1'%col]=checkt['%s_5'%col]/checkt['%s_1'%col]
        checkt['%s_4_3'%col]=checkt['%s_4'%col]/checkt['%s_3'%col]
        checkt['%s_4_2'%col]=checkt['%s_4'%col]/checkt['%s_2'%col]
        checkt['%s_4_1'%col]=checkt['%s_4'%col]/checkt['%s_1'%col]
        checkt['%s_3_2'%col]=checkt['%s_3'%col]/checkt['%s_2'%col]
        checkt['%s_3_1'%col]=checkt['%s_3'%col]/checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]

    for col in ['天数差','公里数差','日均差']:
        checkt['%s_mean'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].mean(1)
        checkt['%s_med'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].median(1)
        checkt['%s_min'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].min(1)
        checkt['%s_max'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].max(1)
        
        checkt['%s_7_6_diff'%col]=checkt['%s_7'%col]-checkt['%s_6'%col]
        checkt['%s_7_5_diff'%col]=checkt['%s_7'%col]-checkt['%s_5'%col]
        checkt['%s_7_4_diff'%col]=checkt['%s_7'%col]-checkt['%s_4'%col]
        checkt['%s_7_3_diff'%col]=checkt['%s_7'%col]-checkt['%s_3'%col]
        checkt['%s_7_2_diff'%col]=checkt['%s_7'%col]-checkt['%s_2'%col]
        checkt['%s_7_1_diff'%col]=checkt['%s_7'%col]-checkt['%s_1'%col]
        checkt['%s_6_5_diff'%col]=checkt['%s_6'%col]-checkt['%s_5'%col]
        checkt['%s_6_4_diff'%col]=checkt['%s_6'%col]-checkt['%s_4'%col]
        checkt['%s_6_3_diff'%col]=checkt['%s_6'%col]-checkt['%s_3'%col]
        checkt['%s_6_2_diff'%col]=checkt['%s_6'%col]-checkt['%s_2'%col]
        checkt['%s_6_1_diff'%col]=checkt['%s_6'%col]-checkt['%s_1'%col]
        checkt['%s_5_4_diff'%col]=checkt['%s_5'%col]-checkt['%s_4'%col]
        checkt['%s_5_3_diff'%col]=checkt['%s_5'%col]-checkt['%s_3'%col]
        checkt['%s_5_2_diff'%col]=checkt['%s_5'%col]-checkt['%s_2'%col]
        checkt['%s_5_1_diff'%col]=checkt['%s_5'%col]-checkt['%s_1'%col]
        checkt['%s_4_3_diff'%col]=checkt['%s_4'%col]-checkt['%s_3'%col]
        checkt['%s_4_2_diff'%col]=checkt['%s_4'%col]-checkt['%s_2'%col]
        checkt['%s_4_1_diff'%col]=checkt['%s_4'%col]-checkt['%s_1'%col]
        checkt['%s_3_2_diff'%col]=checkt['%s_3'%col]-checkt['%s_2'%col]
        checkt['%s_3_1_diff'%col]=checkt['%s_3'%col]-checkt['%s_1'%col]
        checkt['%s_2_1_diff'%col]=checkt['%s_2'%col]-checkt['%s_1'%col]
        
        checkt['%s_7_6'%col]=checkt['%s_7'%col]/checkt['%s_6'%col]
        checkt['%s_7_5'%col]=checkt['%s_7'%col]/checkt['%s_5'%col]
        checkt['%s_7_4'%col]=checkt['%s_7'%col]/checkt['%s_4'%col]
        checkt['%s_7_3'%col]=checkt['%s_7'%col]/checkt['%s_3'%col]
        checkt['%s_7_2'%col]=checkt['%s_7'%col]/checkt['%s_2'%col]
        checkt['%s_7_1'%col]=checkt['%s_7'%col]/checkt['%s_1'%col]
        checkt['%s_6_5'%col]=checkt['%s_6'%col]/checkt['%s_5'%col]
        checkt['%s_6_4'%col]=checkt['%s_6'%col]/checkt['%s_4'%col]
        checkt['%s_6_3'%col]=checkt['%s_6'%col]/checkt['%s_3'%col]
        checkt['%s_6_2'%col]=checkt['%s_6'%col]/checkt['%s_2'%col]
        checkt['%s_6_1'%col]=checkt['%s_6'%col]/checkt['%s_1'%col]
        checkt['%s_5_4'%col]=checkt['%s_5'%col]/checkt['%s_4'%col]
        checkt['%s_5_3'%col]=checkt['%s_5'%col]/checkt['%s_3'%col]
        checkt['%s_5_3'%col]=checkt['%s_5'%col]/checkt['%s_2'%col]
        checkt['%s_5_1'%col]=checkt['%s_5'%col]/checkt['%s_1'%col]
        checkt['%s_4_3'%col]=checkt['%s_4'%col]/checkt['%s_3'%col]
        checkt['%s_4_2'%col]=checkt['%s_4'%col]/checkt['%s_2'%col]
        checkt['%s_4_1'%col]=checkt['%s_4'%col]/checkt['%s_1'%col]
        checkt['%s_3_2'%col]=checkt['%s_3'%col]/checkt['%s_2'%col]
        checkt['%s_3_1'%col]=checkt['%s_3'%col]/checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]
        
        checkt['%s_7_6_inc'%col]=checkt['%s_7_6_diff'%col]/checkt['%s_6'%col]
        checkt['%s_7_5_inc'%col]=checkt['%s_7_5_diff'%col]/checkt['%s_5'%col]
        checkt['%s_7_4_inc'%col]=checkt['%s_7_4_diff'%col]/checkt['%s_4'%col]
        checkt['%s_7_3_inc'%col]=checkt['%s_7_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_7_2_inc'%col]=checkt['%s_7_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_7_1_inc'%col]=checkt['%s_7_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_6_5_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_5'%col]
        checkt['%s_6_4_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_4'%col]
        checkt['%s_6_3_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_3'%col]
        checkt['%s_6_2_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_2'%col]
        checkt['%s_6_1_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_1'%col]
        checkt['%s_5_4_inc'%col]=checkt['%s_5_4_diff'%col]/checkt['%s_4'%col]
        checkt['%s_5_3_inc'%col]=checkt['%s_5_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_5_2_inc'%col]=checkt['%s_5_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_5_1_inc'%col]=checkt['%s_5_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_4_3_inc'%col]=checkt['%s_4_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_4_2_inc'%col]=checkt['%s_4_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_4_1_inc'%col]=checkt['%s_4_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_3_2_inc'%col]=checkt['%s_3_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_3_1_inc'%col]=checkt['%s_3_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_2_1_inc'%col]=checkt['%s_2_1_diff'%col]/checkt['%s_1'%col]
    
    for col in ['天数','公里数','日均']:
        checkt['%s比_2_2'%col]=checkt['%s_2'%col]/checkt['%s差_2'%col]
        checkt['%s比_2_1'%col]=checkt['%s_2'%col]/checkt['%s差_1'%col]
    
        checkt['%s比_3_1'%col]=checkt['%s_3'%col]/checkt['%s差_1'%col]
        checkt['%s比_3_2'%col]=checkt['%s_3'%col]/checkt['%s差_2'%col]
        checkt['%s比_3_3'%col]=checkt['%s_3'%col]/checkt['%s差_3'%col]
    
        checkt['%s比_4_1'%col]=checkt['%s_4'%col]/checkt['%s差_1'%col]
        checkt['%s比_4_2'%col]=checkt['%s_4'%col]/checkt['%s差_2'%col]
        checkt['%s比_4_3'%col]=checkt['%s_4'%col]/checkt['%s差_3'%col]
        checkt['%s比_4_4'%col]=checkt['%s_4'%col]/checkt['%s差_4'%col]
    
        checkt['%s比_5_5'%col]=checkt['%s_5'%col]/checkt['%s差_5'%col]
        checkt['%s比_5_4'%col]=checkt['%s_5'%col]/checkt['%s差_4'%col]
        checkt['%s比_5_3'%col]=checkt['%s_5'%col]/checkt['%s差_3'%col]
        checkt['%s比_5_2'%col]=checkt['%s_5'%col]/checkt['%s差_2'%col]
        checkt['%s比_5_1'%col]=checkt['%s_5'%col]/checkt['%s差_1'%col]
    
        checkt['%s比_6_6'%col]=checkt['%s_6'%col]/checkt['%s差_6'%col]
        checkt['%s比_6_5'%col]=checkt['%s_6'%col]/checkt['%s差_5'%col]
        checkt['%s比_6_4'%col]=checkt['%s_6'%col]/checkt['%s差_4'%col]
        checkt['%s比_6_3'%col]=checkt['%s_6'%col]/checkt['%s差_3'%col]
        checkt['%s比_6_2'%col]=checkt['%s_6'%col]/checkt['%s差_2'%col]
        checkt['%s比_6_1'%col]=checkt['%s_6'%col]/checkt['%s差_1'%col]
    
        checkt['%s比_7_1'%col]=checkt['%s_7'%col]/checkt['%s差_1'%col]
        checkt['%s比_7_2'%col]=checkt['%s_7'%col]/checkt['%s差_2'%col]
        checkt['%s比_7_3'%col]=checkt['%s_7'%col]/checkt['%s差_3'%col]
        checkt['%s比_7_4'%col]=checkt['%s_7'%col]/checkt['%s差_4'%col]
        checkt['%s比_7_5'%col]=checkt['%s_7'%col]/checkt['%s差_5'%col]
        checkt['%s比_7_6'%col]=checkt['%s_7'%col]/checkt['%s差_6'%col]
        
#     for col in ['天数差_1','公里数差_1','日均差_1']:
#         del checkt[col]
    
    print(checkt.shape)
    print(checkt['VIN'].nunique())
    return checkt


# In[146]:


def get_time_feat7(checkt,time_feat1,the_baoyang_id):
    end_id=the_baoyang_id-1

#     for i in range(1,the_baoyang_id):
#         checkt['日均_%d'%i]=checkt['公里数_%d'%i]/checkt['天数_%d'%i] #整段日均

    for col in ['天数','公里数','日均']:
        checkt['%s_6_5'%col]=checkt['%s_6'%col]/checkt['%s_5'%col]
        checkt['%s_6_4'%col]=checkt['%s_6'%col]/checkt['%s_4'%col]
        checkt['%s_6_3'%col]=checkt['%s_6'%col]/checkt['%s_3'%col]
        checkt['%s_6_2'%col]=checkt['%s_6'%col]/checkt['%s_2'%col]
        checkt['%s_6_1'%col]=checkt['%s_6'%col]/checkt['%s_1'%col]
        checkt['%s_5_4'%col]=checkt['%s_5'%col]/checkt['%s_4'%col]
        checkt['%s_5_3'%col]=checkt['%s_5'%col]/checkt['%s_3'%col]
        checkt['%s_5_2'%col]=checkt['%s_5'%col]/checkt['%s_2'%col]
        checkt['%s_5_1'%col]=checkt['%s_5'%col]/checkt['%s_1'%col]
        checkt['%s_4_3'%col]=checkt['%s_4'%col]/checkt['%s_3'%col]
        checkt['%s_4_2'%col]=checkt['%s_4'%col]/checkt['%s_2'%col]
        checkt['%s_4_1'%col]=checkt['%s_4'%col]/checkt['%s_1'%col]
        checkt['%s_3_2'%col]=checkt['%s_3'%col]/checkt['%s_2'%col]
        checkt['%s_3_1'%col]=checkt['%s_3'%col]/checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]

    for col in ['天数差','公里数差','日均差']:
        checkt['%s_mean'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].mean(1)
        checkt['%s_med'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].median(1)
        checkt['%s_min'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].min(1)
        checkt['%s_max'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].max(1)
        
        checkt['%s_6_5_diff'%col]=checkt['%s_6'%col]-checkt['%s_5'%col]
        checkt['%s_6_4_diff'%col]=checkt['%s_6'%col]-checkt['%s_4'%col]
        checkt['%s_6_3_diff'%col]=checkt['%s_6'%col]-checkt['%s_3'%col]
        checkt['%s_6_2_diff'%col]=checkt['%s_6'%col]-checkt['%s_2'%col]
        checkt['%s_6_1_diff'%col]=checkt['%s_6'%col]-checkt['%s_1'%col]
        checkt['%s_5_4_diff'%col]=checkt['%s_5'%col]-checkt['%s_4'%col]
        checkt['%s_5_3_diff'%col]=checkt['%s_5'%col]-checkt['%s_3'%col]
        checkt['%s_5_2_diff'%col]=checkt['%s_5'%col]-checkt['%s_2'%col]
        checkt['%s_5_1_diff'%col]=checkt['%s_5'%col]-checkt['%s_1'%col]
        checkt['%s_4_3_diff'%col]=checkt['%s_4'%col]-checkt['%s_3'%col]
        checkt['%s_4_2_diff'%col]=checkt['%s_4'%col]-checkt['%s_2'%col]
        checkt['%s_4_1_diff'%col]=checkt['%s_4'%col]-checkt['%s_1'%col]
        checkt['%s_3_2_diff'%col]=checkt['%s_3'%col]-checkt['%s_2'%col]
        checkt['%s_3_1_diff'%col]=checkt['%s_3'%col]-checkt['%s_1'%col]
        checkt['%s_2_1_diff'%col]=checkt['%s_2'%col]-checkt['%s_1'%col]
        
        checkt['%s_6_5'%col]=checkt['%s_6'%col]/checkt['%s_5'%col]
        checkt['%s_6_4'%col]=checkt['%s_6'%col]/checkt['%s_4'%col]
        checkt['%s_6_3'%col]=checkt['%s_6'%col]/checkt['%s_3'%col]
        checkt['%s_6_2'%col]=checkt['%s_6'%col]/checkt['%s_2'%col]
        checkt['%s_6_1'%col]=checkt['%s_6'%col]/checkt['%s_1'%col]
        checkt['%s_5_4'%col]=checkt['%s_5'%col]/checkt['%s_4'%col]
        checkt['%s_5_3'%col]=checkt['%s_5'%col]/checkt['%s_3'%col]
        checkt['%s_5_3'%col]=checkt['%s_5'%col]/checkt['%s_2'%col]
        checkt['%s_5_1'%col]=checkt['%s_5'%col]/checkt['%s_1'%col]
        checkt['%s_4_3'%col]=checkt['%s_4'%col]/checkt['%s_3'%col]
        checkt['%s_4_2'%col]=checkt['%s_4'%col]/checkt['%s_2'%col]
        checkt['%s_4_1'%col]=checkt['%s_4'%col]/checkt['%s_1'%col]
        checkt['%s_3_2'%col]=checkt['%s_3'%col]/checkt['%s_2'%col]
        checkt['%s_3_1'%col]=checkt['%s_3'%col]/checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]
        
        checkt['%s_6_5_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_5'%col]
        checkt['%s_6_4_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_4'%col]
        checkt['%s_6_3_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_3'%col]
        checkt['%s_6_2_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_2'%col]
        checkt['%s_6_1_inc'%col]=checkt['%s_6_5_diff'%col]/checkt['%s_1'%col]
        checkt['%s_5_4_inc'%col]=checkt['%s_5_4_diff'%col]/checkt['%s_4'%col]
        checkt['%s_5_3_inc'%col]=checkt['%s_5_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_5_2_inc'%col]=checkt['%s_5_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_5_1_inc'%col]=checkt['%s_5_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_4_3_inc'%col]=checkt['%s_4_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_4_2_inc'%col]=checkt['%s_4_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_4_1_inc'%col]=checkt['%s_4_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_3_2_inc'%col]=checkt['%s_3_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_3_1_inc'%col]=checkt['%s_3_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_2_1_inc'%col]=checkt['%s_2_1_diff'%col]/checkt['%s_1'%col]
    
    for col in ['天数','公里数','日均']:
        checkt['%s比_2_2'%col]=checkt['%s_2'%col]/checkt['%s差_2'%col]
        checkt['%s比_2_1'%col]=checkt['%s_2'%col]/checkt['%s差_1'%col]
    
        checkt['%s比_3_1'%col]=checkt['%s_3'%col]/checkt['%s差_1'%col]
        checkt['%s比_3_2'%col]=checkt['%s_3'%col]/checkt['%s差_2'%col]
        checkt['%s比_3_3'%col]=checkt['%s_3'%col]/checkt['%s差_3'%col]
    
        checkt['%s比_4_1'%col]=checkt['%s_4'%col]/checkt['%s差_1'%col]
        checkt['%s比_4_2'%col]=checkt['%s_4'%col]/checkt['%s差_2'%col]
        checkt['%s比_4_3'%col]=checkt['%s_4'%col]/checkt['%s差_3'%col]
        checkt['%s比_4_4'%col]=checkt['%s_4'%col]/checkt['%s差_4'%col]
    
        checkt['%s比_5_5'%col]=checkt['%s_5'%col]/checkt['%s差_5'%col]
        checkt['%s比_5_4'%col]=checkt['%s_5'%col]/checkt['%s差_4'%col]
        checkt['%s比_5_3'%col]=checkt['%s_5'%col]/checkt['%s差_3'%col]
        checkt['%s比_5_2'%col]=checkt['%s_5'%col]/checkt['%s差_2'%col]
        checkt['%s比_5_1'%col]=checkt['%s_5'%col]/checkt['%s差_1'%col]
    
        checkt['%s比_6_6'%col]=checkt['%s_6'%col]/checkt['%s差_6'%col]
        checkt['%s比_6_5'%col]=checkt['%s_6'%col]/checkt['%s差_5'%col]
        checkt['%s比_6_4'%col]=checkt['%s_6'%col]/checkt['%s差_4'%col]
        checkt['%s比_6_3'%col]=checkt['%s_6'%col]/checkt['%s差_3'%col]
        checkt['%s比_6_2'%col]=checkt['%s_6'%col]/checkt['%s差_2'%col]
        checkt['%s比_6_1'%col]=checkt['%s_6'%col]/checkt['%s差_1'%col]
        
#     for col in ['天数差_1','公里数差_1','日均差_1']:
#         del checkt[col]
    
    print(checkt.shape)
    print(checkt['VIN'].nunique())
    return checkt


# In[147]:


def get_time_feat6(checkt,time_feat1,the_baoyang_id):
    end_id=the_baoyang_id-1

#     for i in range(1,the_baoyang_id):
#         checkt['日均_%d'%i]=checkt['公里数_%d'%i]/checkt['天数_%d'%i] #整段日均

    for col in ['天数','公里数','日均']:
        checkt['%s_5_4'%col]=checkt['%s_5'%col]/checkt['%s_4'%col]
        checkt['%s_5_3'%col]=checkt['%s_5'%col]/checkt['%s_3'%col]
        checkt['%s_5_2'%col]=checkt['%s_5'%col]/checkt['%s_2'%col]
        checkt['%s_5_1'%col]=checkt['%s_5'%col]/checkt['%s_1'%col]
        checkt['%s_4_3'%col]=checkt['%s_4'%col]/checkt['%s_3'%col]
        checkt['%s_4_2'%col]=checkt['%s_4'%col]/checkt['%s_2'%col]
        checkt['%s_4_1'%col]=checkt['%s_4'%col]/checkt['%s_1'%col]
        checkt['%s_3_2'%col]=checkt['%s_3'%col]/checkt['%s_2'%col]
        checkt['%s_3_1'%col]=checkt['%s_3'%col]/checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]

    for col in ['天数差','公里数差','日均差']:
        checkt['%s_mean'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].mean(1)
        checkt['%s_med'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].median(1)
        checkt['%s_min'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].min(1)
        checkt['%s_max'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].max(1)
        
        checkt['%s_5_4_diff'%col]=checkt['%s_5'%col]-checkt['%s_4'%col]
        checkt['%s_5_3_diff'%col]=checkt['%s_5'%col]-checkt['%s_3'%col]
        checkt['%s_5_2_diff'%col]=checkt['%s_5'%col]-checkt['%s_2'%col]
        checkt['%s_5_1_diff'%col]=checkt['%s_5'%col]-checkt['%s_1'%col]
        checkt['%s_4_3_diff'%col]=checkt['%s_4'%col]-checkt['%s_3'%col]
        checkt['%s_4_2_diff'%col]=checkt['%s_4'%col]-checkt['%s_2'%col]
        checkt['%s_4_1_diff'%col]=checkt['%s_4'%col]-checkt['%s_1'%col]
        checkt['%s_3_2_diff'%col]=checkt['%s_3'%col]-checkt['%s_2'%col]
        checkt['%s_3_1_diff'%col]=checkt['%s_3'%col]-checkt['%s_1'%col]
        checkt['%s_2_1_diff'%col]=checkt['%s_2'%col]-checkt['%s_1'%col]
        
        checkt['%s_5_4'%col]=checkt['%s_5'%col]/checkt['%s_4'%col]
        checkt['%s_5_3'%col]=checkt['%s_5'%col]/checkt['%s_3'%col]
        checkt['%s_5_3'%col]=checkt['%s_5'%col]/checkt['%s_2'%col]
        checkt['%s_5_1'%col]=checkt['%s_5'%col]/checkt['%s_1'%col]
        checkt['%s_4_3'%col]=checkt['%s_4'%col]/checkt['%s_3'%col]
        checkt['%s_4_2'%col]=checkt['%s_4'%col]/checkt['%s_2'%col]
        checkt['%s_4_1'%col]=checkt['%s_4'%col]/checkt['%s_1'%col]
        checkt['%s_3_2'%col]=checkt['%s_3'%col]/checkt['%s_2'%col]
        checkt['%s_3_1'%col]=checkt['%s_3'%col]/checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]
        
        checkt['%s_5_4_inc'%col]=checkt['%s_5_4_diff'%col]/checkt['%s_4'%col]
        checkt['%s_5_3_inc'%col]=checkt['%s_5_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_5_2_inc'%col]=checkt['%s_5_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_5_1_inc'%col]=checkt['%s_5_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_4_3_inc'%col]=checkt['%s_4_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_4_2_inc'%col]=checkt['%s_4_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_4_1_inc'%col]=checkt['%s_4_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_3_2_inc'%col]=checkt['%s_3_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_3_1_inc'%col]=checkt['%s_3_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_2_1_inc'%col]=checkt['%s_2_1_diff'%col]/checkt['%s_1'%col]
    
    for col in ['天数','公里数','日均']:
        checkt['%s比_2_2'%col]=checkt['%s_2'%col]/checkt['%s差_2'%col]
        checkt['%s比_2_1'%col]=checkt['%s_2'%col]/checkt['%s差_1'%col]
    
        checkt['%s比_3_1'%col]=checkt['%s_3'%col]/checkt['%s差_1'%col]
        checkt['%s比_3_2'%col]=checkt['%s_3'%col]/checkt['%s差_2'%col]
        checkt['%s比_3_3'%col]=checkt['%s_3'%col]/checkt['%s差_3'%col]
    
        checkt['%s比_4_1'%col]=checkt['%s_4'%col]/checkt['%s差_1'%col]
        checkt['%s比_4_2'%col]=checkt['%s_4'%col]/checkt['%s差_2'%col]
        checkt['%s比_4_3'%col]=checkt['%s_4'%col]/checkt['%s差_3'%col]
        checkt['%s比_4_4'%col]=checkt['%s_4'%col]/checkt['%s差_4'%col]
    
        checkt['%s比_5_5'%col]=checkt['%s_5'%col]/checkt['%s差_5'%col]
        checkt['%s比_5_4'%col]=checkt['%s_5'%col]/checkt['%s差_4'%col]
        checkt['%s比_5_3'%col]=checkt['%s_5'%col]/checkt['%s差_3'%col]
        checkt['%s比_5_2'%col]=checkt['%s_5'%col]/checkt['%s差_2'%col]
        checkt['%s比_5_1'%col]=checkt['%s_5'%col]/checkt['%s差_1'%col]
    
#     for col in ['天数差_1','公里数差_1','日均差_1']:
#         del checkt[col]
    
    print(checkt.shape)
    print(checkt['VIN'].nunique())
    return checkt


# In[148]:


def get_time_feat5(checkt,time_feat1,the_baoyang_id):
    end_id=the_baoyang_id-1

    for col in ['天数','公里数','日均']:
        checkt['%s_4_3'%col]=checkt['%s_4'%col]/checkt['%s_3'%col]
        checkt['%s_4_2'%col]=checkt['%s_4'%col]/checkt['%s_2'%col]
        checkt['%s_4_1'%col]=checkt['%s_4'%col]/checkt['%s_1'%col]
        checkt['%s_3_2'%col]=checkt['%s_3'%col]/checkt['%s_2'%col]
        checkt['%s_3_1'%col]=checkt['%s_3'%col]/checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]

    for col in ['天数差','公里数差','日均差']:
        checkt['%s_mean'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].mean(1)
        checkt['%s_med'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].median(1)
        checkt['%s_min'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].min(1)
        checkt['%s_max'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].max(1)
        
        checkt['%s_4_3_diff'%col]=checkt['%s_4'%col]-checkt['%s_3'%col]
        checkt['%s_4_2_diff'%col]=checkt['%s_4'%col]-checkt['%s_2'%col]
        checkt['%s_4_1_diff'%col]=checkt['%s_4'%col]-checkt['%s_1'%col]
        checkt['%s_3_2_diff'%col]=checkt['%s_3'%col]-checkt['%s_2'%col]
        checkt['%s_3_1_diff'%col]=checkt['%s_3'%col]-checkt['%s_1'%col]
        checkt['%s_2_1_diff'%col]=checkt['%s_2'%col]-checkt['%s_1'%col]
        
        checkt['%s_4_3'%col]=checkt['%s_4'%col]/checkt['%s_3'%col]
        checkt['%s_4_2'%col]=checkt['%s_4'%col]/checkt['%s_2'%col]
        checkt['%s_4_1'%col]=checkt['%s_4'%col]/checkt['%s_1'%col]
        checkt['%s_3_2'%col]=checkt['%s_3'%col]/checkt['%s_2'%col]
        checkt['%s_3_1'%col]=checkt['%s_3'%col]/checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]
        
        checkt['%s_4_3_inc'%col]=checkt['%s_4_3_diff'%col]/checkt['%s_3'%col]
        checkt['%s_4_2_inc'%col]=checkt['%s_4_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_4_1_inc'%col]=checkt['%s_4_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_3_2_inc'%col]=checkt['%s_3_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_3_1_inc'%col]=checkt['%s_3_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_2_1_inc'%col]=checkt['%s_2_1_diff'%col]/checkt['%s_1'%col]
    
    for col in ['天数','公里数','日均']:
        checkt['%s比_2_2'%col]=checkt['%s_2'%col]/checkt['%s差_2'%col]
        checkt['%s比_2_1'%col]=checkt['%s_2'%col]/checkt['%s差_1'%col]
    
        checkt['%s比_3_1'%col]=checkt['%s_3'%col]/checkt['%s差_1'%col]
        checkt['%s比_3_2'%col]=checkt['%s_3'%col]/checkt['%s差_2'%col]
        checkt['%s比_3_3'%col]=checkt['%s_3'%col]/checkt['%s差_3'%col]
    
        checkt['%s比_4_1'%col]=checkt['%s_4'%col]/checkt['%s差_1'%col]
        checkt['%s比_4_2'%col]=checkt['%s_4'%col]/checkt['%s差_2'%col]
        checkt['%s比_4_3'%col]=checkt['%s_4'%col]/checkt['%s差_3'%col]
        checkt['%s比_4_4'%col]=checkt['%s_4'%col]/checkt['%s差_4'%col]
    
#     for col in ['天数差_1','公里数差_1','日均差_1']:
#         del checkt[col]
    
    print(checkt.shape)
    print(checkt['VIN'].nunique())
    return checkt


# In[149]:


def get_time_feat4(checkt,time_feat1,the_baoyang_id):
    end_id=the_baoyang_id-1

    for col in ['天数','公里数','日均']:
        checkt['%s_3_2'%col]=checkt['%s_3'%col]/checkt['%s_2'%col]
        checkt['%s_3_1'%col]=checkt['%s_3'%col]/checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]

    for col in ['天数差','公里数差','日均差']:
        checkt['%s_mean'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].mean(1)
        checkt['%s_med'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].median(1)
        checkt['%s_min'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].min(1)
        checkt['%s_max'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].max(1)
    
        checkt['%s_3_2_diff'%col]=checkt['%s_3'%col]-checkt['%s_2'%col]
        checkt['%s_3_1_diff'%col]=checkt['%s_3'%col]-checkt['%s_1'%col]
        checkt['%s_2_1_diff'%col]=checkt['%s_2'%col]-checkt['%s_1'%col]
        
        checkt['%s_3_2'%col]=checkt['%s_3'%col]/checkt['%s_2'%col]
        checkt['%s_3_1'%col]=checkt['%s_3'%col]/checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]
        
        checkt['%s_3_2_inc'%col]=checkt['%s_3_2_diff'%col]/checkt['%s_2'%col]
        checkt['%s_3_1_inc'%col]=checkt['%s_3_1_diff'%col]/checkt['%s_1'%col]
        checkt['%s_2_1_inc'%col]=checkt['%s_2_1_diff'%col]/checkt['%s_1'%col]

    for col in ['天数','公里数','日均']:
        checkt['%s比_2_2'%col]=checkt['%s_2'%col]/checkt['%s差_2'%col]
        checkt['%s比_2_1'%col]=checkt['%s_2'%col]/checkt['%s差_1'%col]
    
        checkt['%s比_3_1'%col]=checkt['%s_3'%col]/checkt['%s差_1'%col]
        checkt['%s比_3_2'%col]=checkt['%s_3'%col]/checkt['%s差_2'%col]
        checkt['%s比_3_3'%col]=checkt['%s_3'%col]/checkt['%s差_3'%col]
    
#     for col in ['天数差_1','公里数差_1','日均差_1']:
#         del checkt[col]
    
    print(checkt.shape)
    print(checkt['VIN'].nunique())
    return checkt


# In[150]:


def get_time_feat3(checkt,time_feat1,the_baoyang_id):
    end_id=the_baoyang_id-1
    
    for col in ['天数','公里数','日均']:
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]

    for col in ['天数差','公里数差','日均差']:
        checkt['%s_mean'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].mean(1)
#         checkt['%s_min'%col]=checkt.loc[:,'%s_1'%col:'%s_%d'%(col,end_id)].min(1)
        
        checkt['%s_2_1_diff'%col]=checkt['%s_2'%col]-checkt['%s_1'%col]
        checkt['%s_2_1'%col]=checkt['%s_2'%col]/checkt['%s_1'%col]
        checkt['%s_2_1_inc'%col]=checkt['%s_2_1_diff'%col]/checkt['%s_1'%col]

    for col in ['天数','公里数','日均']:
        checkt['%s比_2_2'%col]=checkt['%s_2'%col]/checkt['%s差_2'%col]
        checkt['%s比_2_1'%col]=checkt['%s_2'%col]/checkt['%s差_1'%col]

#     for col in ['天数差_1','公里数差_1','日均差_1']:
#         del checkt[col]

    print(checkt.shape)
    print(checkt['VIN'].nunique())
    return checkt


# In[151]:


def get_time_feat2(checkt,time_feat1,the_baoyang_id):
    vin_cols=['会员卡类型', '性别', '一级车系', 'buy_year', '会员卡年龄', '车龄bin',
              '省份', '城市', '车型档次', '城市级别']
    
    end_id=the_baoyang_id-1
    
    checkt['天数bin']=checkt['天数_1'].apply(lambda x:math.ceil(x/30))
    checkt['公里数bin']=checkt['公里数_1'].apply(lambda x:math.ceil(x/1000))
    checkt['日均bin']=checkt['日均_1'].apply(lambda x:math.ceil(x/10))
    
    for col in vin_cols:
        tmp=checkt[['VIN',col,'天数bin','公里数bin','日均bin']].drop_duplicates()
        tmpp=tmp.groupby([col,'天数bin'])['VIN'].nunique().reset_index(name='%s_%s_人数'%(col,'天数'))
        tmpp['%s_%s_人数占比'%(col,'天数')]=tmpp['%s_%s_人数'%(col,'天数')]/tmp['VIN'].nunique()
        checkt=checkt.merge(tmpp,on=[col,'天数bin'],how='left')
        gc.collect()
        
        tmpp=tmp.groupby([col,'公里数bin'])['VIN'].nunique().reset_index(name='%s_%s_人数'%(col,'公里数'))
        tmpp['%s_%s_人数占比'%(col,'公里数')]=tmpp['%s_%s_人数'%(col,'公里数')]/tmp['VIN'].nunique()
        checkt=checkt.merge(tmpp,on=[col,'公里数bin'],how='left')
        gc.collect()
        
        tmpp=tmp.groupby([col,'日均bin'])['VIN'].nunique().reset_index(name='%s_%s_人数'%(col,'日均'))
        tmpp['%s_%s_人数占比'%(col,'日均')]=tmpp['%s_%s_人数'%(col,'日均')]/tmp['VIN'].nunique()
        checkt=checkt.merge(tmpp,on=[col,'日均bin'],how='left')
    
#    #即使是相同col下的一群人,他们的保养习惯也非常大,所以这样做group并无法分群体
#     for col in vin_cols:
#         thedf=checkt[[col]+['天数_1','公里数_1','日均_1']].drop_duplicates()
#         tmp=thedf.groupby(col)['天数_1','公里数_1','日均_1'].agg({'min','max','mean','std'})
#         tmp.columns=['_'.join(x) for x in tmp.columns.tolist()]
#         tmp.columns=['%s_%s'%(col,x) for x in list(tmp.columns)]
#         tmp=tmp.reset_index()
#         checkt=checkt.merge(tmp,on=[col],how='left')

#     for col in ['天数差_1','公里数差_1','日均差_1']:
#         del checkt[col]
    print(checkt.shape)
    print(checkt['VIN'].nunique())
    return checkt

In [13]:
def run_level_1(rerun,model_name,batch_id,clf,clf_name,dt,dt_arr,train_mat,n,train_y_mat,y_name,second_train_cols1,base_cols):
    second_train_cols2=[]
    kfold=KFold(n_splits=n,random_state=2020)

    mse_train_score=0
    mae_train_score=0
    r2_train_score=0
    mse_vali_score=0
    mae_vali_score=0
    r2_vali_score=0
        
    for i,(train_idx,vali_idx) in enumerate(kfold.split(train_mat)):
        print('------------------Level 1,batch %d------------------'%i)
        train_x,vali_x=train_mat[train_idx],train_mat[vali_idx]
        train_y,vali_y=train_y_mat[train_idx],train_y_mat[vali_idx]
        
        if rerun=='True': #一二个月以后重新跑模型并保存
            clf.fit(train_x,train_y,eval_metric=clf_name,eval_set=[(train_x,train_y),(vali_x,vali_y)],
                early_stopping_rounds=100,verbose=100)
#             f=open(savep+'level_1_%s_batch_%d_model.pickle'%(model_name,batch_id),'wb')
#             pickle.dump(clf,f)
#             f.close()
        else:
            f=open(savep+'level_1_%s_batch_%d_model.pickle'%(model_name,batch_id),'rb')
            clf=pickle.load(f)
            f.close()

        pred_train=clf.predict(train_x,num_iteration=clf.best_iteration_)
        pred_vali=clf.predict(vali_x,num_iteration=clf.best_iteration_)
        
        print('MSE_train :',mean_squared_error(train_y,pred_train))
        print('MSE_vali :',mean_squared_error(vali_y,pred_vali))
        print('r2_train :',r2_score(train_y,pred_train))
        print('r2_vali :',r2_score(vali_y,pred_vali))
        print('MAE_train :',mean_absolute_error(train_y,pred_train))
        print('MAE_vali :',mean_absolute_error(vali_y,pred_vali))
        mse_train_score+=mean_squared_error(train_y,pred_train)
        mse_vali_score+=mean_squared_error(vali_y,pred_vali)
        r2_train_score+=r2_score(train_y,pred_train)
        r2_vali_score+=r2_score(vali_y,pred_vali)
        mae_train_score+=mean_absolute_error(train_y,pred_train)
        mae_vali_score+=mean_absolute_error(vali_y,pred_vali)
        
        dt['%s_%s_pred_%d'%(clf_name,y_name,i)]=clf.predict(dt_arr,num_iteration=clf.best_iteration_)#保存
        second_train_cols2.append('%s_%s_pred_%d'%(clf_name,y_name,i))
    print('#################第一层结果################')
    print('After %d Kfold, the result is :'%n)
    print('MSE_train :',mse_train_score/n)
    print('MSE_vali :',mse_vali_score/n)
    print('r2_train :',r2_train_score/n)
    print('r2_vali :',r2_vali_score/n)
    print('MAE_train :',mae_train_score/n)
    print('MAE_vali :',mae_vali_score/n)
    print("r2 train超过vali多少:",(r2_train_score/n)-(r2_vali_score/n))
    print('############################################')
    
    second_data=dt[base_cols+second_train_cols1+second_train_cols2]

    return second_data,second_train_cols2


# In[153]:


def run_level_2(rerun,model_name,batch_id,clf1,second_data,second_train_col,n1,y,base_cols):
    
    train=second_data[second_data[y].notnull()]
    test=second_data[second_data[y].isnull()]
    
    train_mat1=train[second_train_col].as_matrix()
    print(train_mat1.shape)
    
    train_y_mat1=train[y].values
    print(train_y_mat1.shape)
    
    test_mat1=test[second_train_col].as_matrix()
    print(test_mat1.shape)
    
    kfold1=KFold(n_splits=n1,random_state=2021)

    ensemble_col=[]
    
    sec_mae_train=0
    sec_mae_vali=0
    sec_mse_train=0
    sec_mse_vali=0
    sec_r2_train=0
    sec_r2_vali=0

    for i,(train_idx,vali_idx) in enumerate(kfold1.split(train_mat1)): 
        print('------------------Level 2, batch %d------------------'%i)
        train_x,vali_x=train_mat1[train_idx],train_mat1[vali_idx]
        train_y,vali_y=train_y_mat1[train_idx],train_y_mat1[vali_idx]
        if rerun=='True':
            clf1.fit(train_x,train_y,eval_metric='rmse',eval_set=[(train_x,train_y),(vali_x,vali_y)],
                early_stopping_rounds=100,verbose=100)
#             f1=open(savep+'level_2_%s_batch_%d_model.pickle'%(model_name,batch_id),'wb')
#             pickle.dump(clf1,f1)
#             f1.close()
        else:
            f1=open(savep+'level_2_%s_batch_%d_model.pickle'%(model_name,batch_id),'rb')
            clf1=pickle.load(f1)
            f1.close()
        
        pred_train=clf1.predict(train_x,num_iteration=clf1.best_iteration_)
        pred_vali=clf1.predict(vali_x,num_iteration=clf1.best_iteration_)
        print('MSE_train :',mean_squared_error(train_y,pred_train))
        print('MSE_vali :',mean_squared_error(vali_y,pred_vali))
        print('r2_train :',r2_score(train_y,pred_train))
        print('r2_vali :',r2_score(vali_y,pred_vali))
        print('MAE_train :',mean_absolute_error(train_y,pred_train))
        print('MAE_vali :',mean_absolute_error(vali_y,pred_vali))
        sec_mse_train+=mean_squared_error(train_y,pred_train)
        sec_mse_vali+=mean_squared_error(vali_y,pred_vali)
        sec_mae_train+=mean_absolute_error(train_y,pred_train)
        sec_mae_vali+=mean_absolute_error(vali_y,pred_vali)
        sec_r2_train+=r2_score(train_y,pred_train)
        sec_r2_vali+=r2_score(vali_y,pred_vali)
            
        train['%s_pred_%d'%(y,i)]=clf1.predict(train_mat1,num_iteration=clf1.best_iteration_)

        test['%s_pred_%d'%(y,i)]=clf1.predict(test_mat1,num_iteration=clf1.best_iteration_)#保存
        ensemble_col.append('%s_pred_%d'%(y,i))

    print('####################第二层后最终结果#########################')
    print('After %d Kfold, the result is :'%n1)
    print('MSE_train :',sec_mse_train/n1)
    print('MSE_vali :',sec_mse_vali/n1)
    print('r2_train :',sec_r2_train/n1)
    print('r2_vali :',sec_r2_vali/n1)
    print('MAE_train :',sec_mae_train/n1)
    print('MAE_vali :',sec_mae_vali/n1)
    print('r2 train 比 vali 过拟合了多少: ',(sec_r2_train/n1)-(sec_r2_vali/n1))
    print('#############################################################')
    
    bad_case_train=train[base_cols+ensemble_col]
    result=test[base_cols+ensemble_col]
    
    eva_lst=[batch_id,sec_mse_train/n1,sec_mse_vali/n1,sec_r2_train/n1,sec_r2_vali/n1,sec_mae_train/n1,sec_mae_vali/n1]

    return bad_case_train,result,eva_lst


def run_one_label(second_train_cols1,base_cols,rerun,batch_id,dt,dt_arr,train_mat,train_y_mat1,y,n,n1):

    clf_mae=lgb.LGBMRegressor(boosting_type='gbdt',objective='regression',num_leaves=82,max_depth=7,
                            subsample=0.70, colsample_bytree=0.70,learning_rate=0.02,n_estimators=2000)
    clf_name_mae='mae'
    if y=='天数差_next':
        model_name='day_mae'
    elif y=='公里数差_next':
        model_name='mile_mae'
    elif y=='日均_next':
        model_name='dayofmile_mae'
    second_data_mae,second_train_cols_mae=run_level_1(rerun,model_name,batch_id,clf_mae,clf_name_mae,dt,dt_arr,train_mat,n,train_y_mat1,y,second_train_cols1,base_cols)
    
    
    clf_mse=lgb.LGBMRegressor(boosting_type='gbdt',objective='regression',num_leaves=82,max_depth=7,
                            subsample=0.70, colsample_bytree=0.70,learning_rate=0.02,n_estimators=2000)
    clf_name_mse='mse'
    if y=='天数差_next':
        model_name='day_mse'
    elif y=='公里数差_next':
        model_name='mile_mse'
    elif y=='日均_next':
        model_name='dayofmile_mse'
    second_data_mse,second_train_cols_mse=run_level_1(rerun,model_name,batch_id,clf_mse,clf_name_mse,dt,dt_arr,train_mat,n,train_y_mat1,y,second_train_cols1,base_cols)
    
    second_data=second_data_mse.merge(second_data_mae,on=second_train_cols1+base_cols,how='left')
    second_train_col=second_train_cols1+second_train_cols_mse+second_train_cols_mae
    clf1=lgb.LGBMRegressor(boosting_type='gbdt',objective='regression',num_leaves=82,max_depth=7,subsample=0.70,
                       colsample_bytree=0.70,learning_rate=0.01,n_estimators=2000)
    if y=='天数差_next':
        model_name1='day'
    elif y=='公里数差_next':
        model_name1='mile'
    elif y=='日均_next':
        model_name1='dayofmile'
    bad_case_train,result,eva_lst=run_level_2(rerun,model_name1,batch_id,clf1,second_data,second_train_col,n1,y,base_cols)
    
    return bad_case_train,result,eva_lst

In [14]:
def get_batch(rerun,savep,dft,the_baoyang_id,baoyang_id_max):
    checkt,train_cols1,train_cols2=get_one_batch_data(dft,the_baoyang_id,baoyang_id_max)
        
    train1=checkt[checkt['天数差_next'].notnull()]
    test1=checkt[checkt['天数差_next'].isnull()]
    
    dt=pd.concat([train1,test1],axis=0)
    
    train_cols=train_cols1+train_cols2
    if '数据集标注' in train_cols:
        train_cols.remove('数据集标注')
    if 'VIN' in train_cols:
        train_cols.remove('VIN')
    print("训练数据的维度是:",len(train_cols))
    print(train_cols)

    train_y_mat1=train1['天数差_next'].values
    print(train_y_mat1.shape)

    train_y_mat2=train1['公里数差_next'].values
    print(train_y_mat2.shape)  
    
    train_y_mat3=train1['日均_next'].values
    print(train_y_mat3.shape)
    
    train_mat=train1[train_cols].values
    print(train_mat.shape)
    
    second_train_cols1=['性别','一级车系','车龄bin','城市','城市级别','车辆类型','使用性质','车主性质']
    
    base_cols=['VIN','购车时间','在店状态','保养状态','人群_一级分类','人群_二级分类',
               '期间最后一次保养天数','期间最后一次保养公里数',
               '天数差_next','公里数差_next','日均_next']

    y1='天数差_next'
    if rerun=='True':
        imp=pd.DataFrame({'features':train_cols})
        clf=lgb.LGBMRegressor(n_estimators=3000)
        clf.fit(train_mat,train_y_mat1,eval_metric='mae',eval_set=[(train_mat,train_y_mat1)],
                early_stopping_rounds=100,verbose=100)
        imp['imp']=clf.feature_importances_
        right_idx_df=imp[imp['imp']>0][['features']]
#         right_idx_df.to_pickle(savep+'day_imp_%d.pkl'%the_baoyang_id)
        right_idx=right_idx_df['features'].unique().tolist()
    else:
        right_idx_df=pd.read_pickle(savep+'day_imp_%d.pkl'%the_baoyang_id)
        right_idx=right_idx_df['features'].unique().tolist()
    
    dt_arr_further=dt[right_idx].values
    train_mat_further=train1[right_idx].values
#     test_mat_further=test1[right_idx].values
    tianshu_train,tianshu_result,tianshu_eva_lst=run_one_label(second_train_cols1,base_cols,rerun,the_baoyang_id,dt,dt_arr_further,train_mat_further,train_y_mat1,y1,5,10)
    tianshu_eva_df=pd.DataFrame(tianshu_eva_lst).T
    tianshu_eva_df.columns=['batch','mae_train','mae_vali','r2_train','r2_vali','mse_train','mse_vali']
    tianshu_eva_df['model']='天数模型'

    
    y2='公里数差_next'
    if rerun=='True':
        imp2=pd.DataFrame({'features':train_cols})
        clf=lgb.LGBMRegressor(n_estimators=3000)
        clf.fit(train_mat,train_y_mat2,eval_metric='mae',eval_set=[(train_mat,train_y_mat2)],
             early_stopping_rounds=100,verbose=100)
        imp2['imp']=clf.feature_importances_
        right_idx_df=imp2[imp2['imp']>0][['features']]
#         right_idx_df.to_pickle(savep+'mile_imp_%d.pkl'%the_baoyang_id)
        right_idx=right_idx_df['features'].unique().tolist()
    else:
        right_idx_df=pd.read_pickle(savep+'mile_imp_%d.pkl'%the_baoyang_id)
        right_idx=right_idx_df['features'].unique().tolist()
        
    dt_arr_further2=dt[right_idx].values
    train_mat_further2=train1[right_idx].values
#     test_mat_further2=test1[right_idx].values
    gongli_train,gongli_result,gongli_eva_lst=run_one_label(second_train_cols1,base_cols,rerun,the_baoyang_id,dt,dt_arr_further2,train_mat_further2,train_y_mat2,y2,5,10)
    gongli_eva_df=pd.DataFrame(gongli_eva_lst).T
    gongli_eva_df.columns=['batch','mae_train','mae_vali','r2_train','r2_vali','mse_train','mse_vali']
    gongli_eva_df['model']='公里数模型'   
    
    
    y3='日均_next'
    if rerun=='True':
        imp3=pd.DataFrame({'features':train_cols})
        clf=lgb.LGBMRegressor(n_estimators=3000)
        clf.fit(train_mat,train_y_mat3,eval_metric='mae',eval_set=[(train_mat,train_y_mat3)],
             early_stopping_rounds=100,verbose=100)
        imp3['imp']=clf.feature_importances_
        right_idx_df=imp3[imp3['imp']>0][['features']]
#         right_idx_df.to_pickle(savep+'dayofmile_imp_%d.pkl'%the_baoyang_id)
        right_idx=right_idx_df['features'].unique().tolist()
    else:
        right_idx_df=pd.read_pickle(savep+'dayofmile_imp_%d.pkl'%the_baoyang_id)
        right_idx=right_idx_df['features'].unique().tolist()
        
    dt_arr_further3=dt[right_idx].values
    train_mat_further3=train1[right_idx].values
    dayofmile_train,dayofmile_result,dayofmile_eva_lst=run_one_label(second_train_cols1,base_cols,rerun,the_baoyang_id,dt,dt_arr_further3,train_mat_further3,train_y_mat3,y3,5,10)
    dayofmile_eva_df=pd.DataFrame(dayofmile_eva_lst).T
    dayofmile_eva_df.columns=['batch','mae_train','mae_vali','r2_train','r2_vali','mse_train','mse_vali']
    dayofmile_eva_df['model']='日均模型'  
    
    same_cols=[x for x in list(gongli_train.columns) if x in list(dayofmile_train.columns)]
    trainf=gongli_train.merge(dayofmile_train,on=same_cols,how='left')
    same_cols=[x for x in list(tianshu_train.columns) if x in list(same_cols)]
    trainf=trainf.merge(tianshu_train,on=same_cols,how='left')
    
    same_cols=[x for x in list(gongli_result.columns) if x in list(dayofmile_result.columns)]
    resultf=gongli_result.merge(dayofmile_result,on=same_cols,how='left')
    same_cols=[x for x in list(tianshu_result.columns) if x in same_cols]
    resultf=resultf.merge(tianshu_result,on=same_cols,how='left')

    eva_df=pd.concat([tianshu_eva_df,gongli_eva_df,dayofmile_eva_df],axis=0)
    
    return trainf,resultf,eva_df

In [15]:
def get_fin_pred(df,n1):

    df['天数差_next_fin']=0
    for i in range(n1):
        df['天数差_next_fin']=df['天数差_next_fin']+df['天数差_next_pred_%d'%(i)]
    df['天数差_next_fin']=df['天数差_next_fin']/n1

    df['公里数差_next_fin']=0
    for i in range(n1):
        df['公里数差_next_fin']=df['公里数差_next_fin']+df['公里数差_next_pred_%d'%(i)]
    df['公里数差_next_fin']=df['公里数差_next_fin']/n1
    
    df['日均_next_fin']=0
    for i in range(n1):
        df['日均_next_fin']=df['日均_next_fin']+df['日均_next_pred_%d'%(i)]
    df['日均_next_fin']=df['日均_next_fin']/n1
    
    return df

In [16]:
def get_fin_pred_mile_dayofmile(df,is_train):
    
    dff=df.copy()
    
    if '天数差_next_fin' in list(dff.columns):
        del dff['天数差_next_fin']

    dff['天数差_next_fin']=dff['公里数差_next_fin']/dff['日均_next_fin']
    dff['天数差']=(dff['天数差_next_fin']-dff['期间最后一次保养天数']).apply(lambda x:math.ceil(x))

    dff['公里数差']=(dff['公里数差_next_fin']-dff['期间最后一次保养公里数']).apply(lambda x:math.ceil(x))
    dff['最终日均']=dff['公里数差']/dff['天数差']
    
    dff['下一次保养公里数']=dff['公里数差_next_fin'].apply(lambda x:math.ceil(x))
    dff['天数差_next_fin']=dff['天数差_next_fin'].apply(lambda x:math.ceil(x))
    dff['购车时间']=pd.to_datetime(dff['购车时间'])
    dff['下一次保养日期']=list(map(lambda x,y:x+relativedelta(days=y),dff['购车时间'],dff['天数差_next_fin']))
    
    dff['最后一次保养日期']=list(map(lambda x,y:x+relativedelta(days=y),dff['购车时间'],dff['期间最后一次保养天数']))
    dff['最后一次保养公里数']=dff['期间最后一次保养公里数']
    
    dff=dff.drop_duplicates().reset_index(drop=True)
    
    return dff

In [17]:
def evaluate(data1,the_baoyang_id,version):
    vins=data1['VIN'].nunique()
    
#     if 'next_date' in list(data1.columns):
#         data1['next_date']=pd.to_datetime(data1['next_date'])
#     if 'last_date' in list(data1.columns):
#         data1['last_date']=pd.to_datetime(data1['last_date'])
    
    if '天数偏差' not in list(data1.columns):
        data1['天数差_next']=data1['天数差_next'].astype(float)
        data1['天数偏差']=(data1['天数差_next']-data1['天数差_next_fin']).apply(lambda x:abs(x))
        
    data1.loc[data1['天数偏差']<=10,'天数偏差人群']='0-10天'
    data1.loc[(data1['天数偏差']>10)&(data1['天数偏差']<=20),'天数偏差人群']='10-20天'
    data1.loc[(data1['天数偏差']>20)&(data1['天数偏差']<=30),'天数偏差人群']='20-30天'
    data1.loc[(data1['天数偏差']>30)&(data1['天数偏差']<=40),'天数偏差人群']='30-40天'
    data1.loc[(data1['天数偏差']>40)&(data1['天数偏差']<=50),'天数偏差人群']='40-50天'
    data1.loc[(data1['天数偏差']>50)&(data1['天数偏差']<=60),'天数偏差人群']='50-60天'
    data1.loc[(data1['天数偏差']>60)&(data1['天数偏差']<=70),'天数偏差人群']='60-70天'
    data1.loc[(data1['天数偏差']>70)&(data1['天数偏差']<=80),'天数偏差人群']='70-80天'
    data1.loc[(data1['天数偏差']>80)&(data1['天数偏差']<=90),'天数偏差人群']='80-90天'
    data1.loc[data1['天数偏差']>90,'天数偏差人群']='90-inf天'
    
    tmp=data1.groupby('天数偏差人群')['VIN'].nunique().reset_index(name='人数')
    tmp['占比']=tmp['人数']/vins
    tmp['idx']=np.arange(len(tmp))
    print(tmp.head(10))

    if '公里数偏差' not in list(data1.columns):
        data1['公里数偏差']=(data1['公里数差_next']-data1['公里数差_next_fin']).apply(abs)
    data1.loc[data1['公里数偏差']<=1000,'公里数偏差人群']='0-1000公里'
    data1.loc[(data1['公里数偏差']>1000)&(data1['公里数偏差']<=2000),'公里数偏差人群']='1000-2000公里'
    data1.loc[(data1['公里数偏差']>2000)&(data1['公里数偏差']<=3000),'公里数偏差人群']='2000-3000公里'
    data1.loc[(data1['公里数偏差']>3000)&(data1['公里数偏差']<=4000),'公里数偏差人群']='3000-4000公里'
    data1.loc[(data1['公里数偏差']>4000)&(data1['公里数偏差']<=5000),'公里数偏差人群']='4000-5000公里'
    data1.loc[(data1['公里数偏差']>5000)&(data1['公里数偏差']<=6000),'公里数偏差人群']='5000-6000公里'
    data1.loc[(data1['公里数偏差']>6000)&(data1['公里数偏差']<=7000),'公里数偏差人群']='6000-7000公里'
    data1.loc[(data1['公里数偏差']>7000)&(data1['公里数偏差']<=8000),'公里数偏差人群']='7000-8000公里'
    data1.loc[(data1['公里数偏差']>8000)&(data1['公里数偏差']<=9000),'公里数偏差人群']='8000-9000公里'
    data1.loc[data1['公里数偏差']>9000,'公里数偏差人群']='9000-公里以上'    
    
    tmpp=data1.groupby('公里数偏差人群')['VIN'].nunique().reset_index(name='人数')
    tmpp['占比']=tmpp['人数']/vins
    tmpp['idx']=np.arange(len(tmpp))
    print(tmpp.head(10))
    
    fin=tmp.merge(tmpp,on='idx',how='left')
    fin['batch']=the_baoyang_id
    fin['version']=version
    
    return fin

In [18]:
def get_fin_pred_day_dayofmile(df,is_train):
    
    dff=df.copy()
    
    if '公里数差_next_fin' in list(dff.columns):
        del dff['公里数差_next_fin']
    
    dff['公里数差_next_fin']=dff['天数差_next_fin']*dff['日均_next_fin']
    dff['公里数差']=(dff['公里数差_next_fin']-dff['期间最后一次保养公里数']).apply(lambda x:math.ceil(x))
    
    dff['天数差']=(dff['天数差_next_fin']-dff['期间最后一次保养天数']).apply(lambda x:math.ceil(x))

    dff['最终日均']=dff['公里数差']/dff['天数差']
    
    dff['下一次保养公里数']=dff['公里数差_next_fin'].apply(lambda x:math.ceil(x))
    dff['天数差_next_fin']=dff['天数差_next_fin'].apply(lambda x:math.ceil(x))
    dff['购车时间']=pd.to_datetime(dff['购车时间'])
    dff['下一次保养日期']=list(map(lambda x,y:x+relativedelta(days=y),dff['购车时间'],dff['天数差_next_fin']))
    
    dff['最后一次保养日期']=list(map(lambda x,y:x+relativedelta(days=y),dff['购车时间'],dff['期间最后一次保养天数']))
    dff['最后一次保养公里数']=dff['期间最后一次保养公里数']
    
    dff=dff.drop_duplicates().reset_index(drop=True)
    
    return dff

In [19]:
def get_fin_pred_day_mile(df,is_train):
    
    dff=df.copy()
    
    dff['公里数差']=(dff['公里数差_next_fin']-dff['期间最后一次保养公里数']).apply(lambda x:math.ceil(x))
    
    dff['天数差']=(dff['天数差_next_fin']-dff['期间最后一次保养天数']).apply(lambda x:math.ceil(x))

    dff['最终日均']=dff['公里数差']/dff['天数差']
    
    dff['下一次保养公里数']=dff['公里数差_next_fin'].apply(lambda x:math.ceil(x))
    dff['天数差_next_fin']=dff['天数差_next_fin'].apply(lambda x:math.ceil(x))
    dff['购车时间']=pd.to_datetime(dff['购车时间'])
    dff['下一次保养日期']=list(map(lambda x,y:x+relativedelta(days=y),dff['购车时间'],dff['天数差_next_fin']))
    
    dff['最后一次保养日期']=list(map(lambda x,y:x+relativedelta(days=y),dff['购车时间'],dff['期间最后一次保养天数']))
    dff['最后一次保养公里数']=dff['期间最后一次保养公里数']
    
    dff=dff.drop_duplicates().reset_index(drop=True)
    
    return dff

In [20]:
# the_baoyang_id=2
# trainf,resultf,eva_df=get_batch(rerun,savep,dft1,the_baoyang_id,baoyang_id_max)
# trainf_check=get_fin_pred(trainf,10)
# cols=['VIN', '购车时间', '在店状态', '保养状态', '人群_一级分类', '人群_二级分类', '期间最后一次保养天数',
#        '期间最后一次保养公里数', '天数差_next', '公里数差_next', '日均_next','天数差_next_fin',
#        '公里数差_next_fin', '日均_next_fin']
# trainf_check_0=trainf_check[cols]
# print(trainf_check_0.shape)
# print(trainf_check_0['VIN'].nunique())

# resultf_check=get_fin_pred(resultf,10)
# cols=['VIN', '购车时间', '在店状态', '保养状态', '人群_一级分类', '人群_二级分类', '期间最后一次保养天数',
#        '期间最后一次保养公里数', '天数差_next_fin', '公里数差_next_fin', '日均_next_fin']
# resultf_check_0=resultf_check[cols]
# print(resultf_check_0.shape)
# print(resultf_check_0['VIN'].nunique())

# eva_df.to_pickle(savep+'%d_模型训练评价指标.pkl'%the_baoyang_id)

# is_train='True'
# trainf_check_1=get_fin_pred_mile_dayofmile(trainf_check_0,is_train)
# version1='mile+dayofmile'
# fin1=evaluate(trainf_check_1,the_baoyang_id,version1)
# fin1.head(50)

# trainf_check_2=get_fin_pred_day_dayofmile(trainf_check_0,is_train)
# version2='day+dayofmile'
# fin2=evaluate(trainf_check_2,the_baoyang_id,version2)
# fin2.head(50)

# trainf_check_3=get_fin_pred_day_mile(trainf_check_0,is_train)
# version3='day+dayofmile'
# fin3=evaluate(trainf_check_3,the_baoyang_id,version3)
# fin3.head(50)

# fin=pd.concat([fin1,fin2,fin3],axis=0)
# fin.to_pickle(savep+'%d_天数偏差.pkl'%the_baoyang_id)

In [1]:
print(datetime.now())

ft=time.time()

#与上一批次差了21万条数据
baoyang=pd.read_pickle(savep+'final_baoyang.pkl')

veh=pd.read_pickle(savep+'vehicle_info.pkl')

rerun='True'
old_baoyang_id_max=0

baoyangdff1,_=get_final_baoyang(baoyang,veh,datap)
dft1=get_all_feat_eng(baoyangdff1,today)

In [ ]:
for the_baoyang_id in range(2,baoyang_id_max+1,1):
    print('-------------------------------%d---------------------------'%the_baoyang_id)
    s1=time.time()
    trainf,resultf,eva_df=get_batch(rerun,savep,dft1,the_baoyang_id,baoyang_id_max)
    trainf_check=get_fin_pred(trainf,10)   
    train_cols=['VIN', '购车时间', '在店状态', '保养状态', '人群_一级分类', '人群_二级分类', '期间最后一次保养天数',
       '期间最后一次保养公里数', '天数差_next', '公里数差_next', '日均_next','天数差_next_fin',
       '公里数差_next_fin', '日均_next_fin']
    trainf_check_0=trainf_check[train_cols]
    
    resultf_check=get_fin_pred(resultf,10)
    test_cols=['VIN', '购车时间', '在店状态', '保养状态', '人群_一级分类', '人群_二级分类',
               '期间最后一次保养天数', '期间最后一次保养公里数', '天数差_next_fin',
               '公里数差_next_fin', '日均_next_fin']
    resultf_check_0=resultf_check[test_cols]
    
    eva_df.to_pickle(savep+'%d_模型训练评价指标.pkl'%the_baoyang_id)

    is_train='True'
    trainf_check_1=get_fin_pred_mile_dayofmile(trainf_check_0,is_train)
    version_1='mile+dayofmile'
    evaluation_1 = evaluate(trainf_check_1,the_baoyang_id,version_1)
    
    trainf_check_2=get_fin_pred_day_dayofmile(trainf_check_0,is_train)
    version_2='day+dayofmile'
    evaluation_2 = evaluate(trainf_check_2,the_baoyang_id,version_2)
    
    trainf_check_3=get_fin_pred_day_mile(trainf_check_0,is_train)
    version_3='day+mile'
    evaluation_3 = evaluate(trainf_check_3,the_baoyang_id,version_3)
    trainf_check_3.to_pickle(savep+'%d_model_train.pkl'%the_baoyang_id)
    
    evaluation = pd.concat([evaluation_1,evaluation_2,evaluation_3],axis=0)
    evaluation.to_pickle(savep+'%d_偏差统计.pkl'%the_baoyang_id)
    
    is_train='False'
    resultf_check_3=get_fin_pred_day_mile(resultf_check,is_train)
    resultf_check_3.to_pickle(savep+'%d_model_result.pkl'%the_baoyang_id)
    
    print('Batch %d run time is: ',(time.time()-s1)/60)

In [ ]:
print('all run time is:',(time.time()-ft)/60)